In [23]:
import pandas as pd 
import os
df = pd.read_csv('fpl-data-stats.csv')
df.describe()

,id,element_type,now_cost,selected_by_percent,gameweek,minutes,shots,SoT,SiB,xG,...,defensive_contribution,xGI,npxGI,xP,total_points,PvsxP,touches,penalty_area_touches,carries_final_third,carries_penalty_area
count,11013.000000,11013.000000,11013.000000,11013.000000,11013.000000,11013.000000,11013.000000,11013.000000,11013.000000,11013.000000,...,11013.000000,11013.000000,11013.000000,11013.000000,11013.000000,11013.000000,4552.000000,4552.000000,11013.000000,11013.000000
mean,369.154000,2.547444,4.956361,2.020476,8.052937,26.639426,0.323527,0.108599,0.218015,0.035313,...,2.084173,0.059121,0.056397,1.215612,1.233361,0.017749,38.499121,1.502197,0.315173,0.119858
std,213.857532,0.834816,1.100609,5.833568,4.275966,37.664690,0.815213,0.387101,0.641490,0.132201,...,3.682147,0.176322,0.165615,2.029076,2.432045,1.460662,24.948349,1.910890,0.843074,0.498252
min,1.000000,1.000000,3.800000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-2.000000,-3.000000,-11.400000,0.000000,0.000000,0.000000,0.000000
25%,184.000000,2.000000,4.300000,0.100000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.000000,0.000000,0.000000,0.000000
50%,368.000000,3.000000,4.700000,0.200000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,36.000000,1.000000,0.000000,0.000000
75%,554.000000,3.000000,5.300000,1.000000,12.000000,69.000000,0.000000,0.000000,0.000000,0.000000,...,3.000000,0.000000,0.000000,2.037000,1.000000,0.000000,55.000000,2.000000,0.000000,0.000000
max,759.000000,4.000000,15.000000,73.000000,15.000000,90.000000,11.000000,5.000000,9.000000,3.500000,...,25.000000,3.500000,2.700000,19.000000,24.000000,15.249000,155.000000,18.000000,9.000000,11.000000


# 1️⃣ Data Loading & Overview {#data-loading--overview}

## 📂 Import Data and Initial Exploration
This section loads the FPL dataset and provides basic information about its structure.

In [24]:
# Dataset Overview and Structure
print("=== DATASET OVERVIEW ===")
print(f"Dataset Shape: {df.shape}")
print(f"Total Records: {df.shape[0]:,}")
print(f"Total Features: {df.shape[1]}")
print("\n=== COLUMN NAMES ===")
print(df.columns.tolist())

print("\n=== DATA TYPES ===")
print(df.dtypes)

print("\n=== BASIC INFO ===")
df.info()

=== DATASET OVERVIEW ===
Dataset Shape: (11013, 37)
Total Records: 11,013
Total Features: 37

=== COLUMN NAMES ===
['id', 'element_type', 'web_name', 'team_name', 'opponent_team_name', 'was_home', 'now_cost', 'selected_by_percent', 'gameweek', 'minutes', 'shots', 'SoT', 'SiB', 'xG', 'npxG', 'G', 'npG', 'key_passes', 'xA', 'A', 'xGC', 'GC', 'xCS', 'CS', 'clearances_blocks_interceptions', 'recoveries', 'tackles', 'defensive_contribution', 'xGI', 'npxGI', 'xP', 'total_points', 'PvsxP', 'touches', 'penalty_area_touches', 'carries_final_third', 'carries_penalty_area']

=== DATA TYPES ===
id                                   int64
element_type                         int64
web_name                            object
team_name                           object
opponent_team_name                  object
was_home                              bool
now_cost                           float64
selected_by_percent                float64
gameweek                             int64
minutes                

In [25]:
# Missing Values Analysis
print("=== MISSING VALUES ANALYSIS ===")
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100

missing_df = pd.DataFrame({
    'Column': missing_values.index,
    'Missing Count': missing_values.values,
    'Missing Percentage': missing_percentage.values
}).sort_values('Missing Count', ascending=False)

# Display only columns with missing values
if missing_df['Missing Count'].sum() > 0:
    print(missing_df[missing_df['Missing Count'] > 0])
else:
    print("No missing values found in the dataset!")

print(f"\nTotal missing values in dataset: {missing_values.sum():,}")
print(f"Percentage of complete records: {((len(df) - missing_values.sum()) / len(df)) * 100:.2f}%")

df = df.drop(columns=['penalty_area_touches', 'touches'])

=== MISSING VALUES ANALYSIS ===
                  Column  Missing Count  Missing Percentage
34  penalty_area_touches           6461            58.66703
33               touches           6461            58.66703

Total missing values in dataset: 12,922
Percentage of complete records: -17.33%


# 2️⃣ Data Cleaning & Processing {#data-cleaning--processing}

## 🧹 Data Quality Assessment and Cleaning
Analyzing missing values, data types, and performing necessary data cleaning operations.

In [26]:
# Separate Numerical and Categorical Variables
import numpy as np

# Identify numerical and categorical columns
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

print("=== VARIABLE TYPES ===")
print(f"Numerical variables ({len(numerical_cols)}): {numerical_cols}")
print(f"\nCategorical variables ({len(categorical_cols)}): {categorical_cols}")

# For categorical variables, show unique values
# For categorical variables, show unique values
print("\n=== CATEGORICAL VARIABLES ANALYSIS ===")
for col in categorical_cols[:10]:  # Show first 10 categorical columns
    unique_count = df[col].nunique()
    print(f"\n{col}:")
    print(f"  - Unique values: {unique_count}")
    if unique_count <= 20:  # Show values if not too many
        # Convert all values to string for sorting (to avoid errors)
        print(f"  - Values: {sorted(df[col].astype(str).unique())}")
    else:
        print(f"  - Top 10 values: {df[col].value_counts().head(10).index.tolist()}")


=== VARIABLE TYPES ===
Numerical variables (31): ['id', 'element_type', 'now_cost', 'selected_by_percent', 'gameweek', 'minutes', 'shots', 'SoT', 'SiB', 'xG', 'npxG', 'G', 'npG', 'key_passes', 'xA', 'A', 'xGC', 'GC', 'xCS', 'CS', 'clearances_blocks_interceptions', 'recoveries', 'tackles', 'defensive_contribution', 'xGI', 'npxGI', 'xP', 'total_points', 'PvsxP', 'carries_final_third', 'carries_penalty_area']

Categorical variables (3): ['web_name', 'team_name', 'opponent_team_name']

=== CATEGORICAL VARIABLES ANALYSIS ===

web_name:
  - Unique values: 738
  - Top 10 values: ['Patterson', "O'Brien", 'Gomez', 'White', 'Barnes', 'Neto', 'Roberts', 'Henderson', 'James', 'Wilson']

team_name:
  - Unique values: 20
  - Values: ['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford', 'Brighton', 'Burnley', 'Chelsea', 'Crystal Palace', 'Everton', 'Fulham', 'Leeds', 'Liverpool', 'Man City', 'Man Utd', 'Newcastle', "Nott'm Forest", 'Spurs', 'Sunderland', 'West Ham', 'Wolves']

opponent_team_name:
  

In [27]:
# Filter useful numerical variables for FPL analysis
print("=== FILTERING USEFUL NUMERICAL VARIABLES ===")

# Define categories of useful variables
core_performance = ['total_points', 'minutes', 'now_cost', 'selected_by_percent']
attacking_metrics = ['G', 'A', 'xG', 'xA', 'shots', 'SoT', 'key_passes']
expected_metrics = ['xG', 'xA', 'xGI', 'npxG', 'npxGI', 'xP']
defensive_metrics = ['CS', 'xCS', 'GC', 'xGC', 'tackles', 'recoveries', 
                    'clearances_blocks_interceptions', 'defensive_contribution']
advanced_metrics = ['PvsxP', 'carries_final_third', 'carries_penalty_area']

# Combine into useful variables list
useful_numerical_vars = list(set(core_performance + attacking_metrics + 
                                expected_metrics + defensive_metrics + advanced_metrics))

# Filter only variables that exist in the dataset
useful_vars_available = [var for var in useful_numerical_vars if var in numerical_cols]

print(f"Original numerical variables: {len(numerical_cols)}")
print(f"Useful numerical variables: {len(useful_vars_available)}")
print(f"Variables removed: {len(numerical_cols) - len(useful_vars_available)}")

print(f"\n=== USEFUL VARIABLES BY CATEGORY ===")
print(f"Core Performance: {[v for v in core_performance if v in useful_vars_available]}")
print(f"Attacking Metrics: {[v for v in attacking_metrics if v in useful_vars_available]}")
print(f"Expected Stats: {[v for v in expected_metrics if v in useful_vars_available]}")
print(f"Defensive Metrics: {[v for v in defensive_metrics if v in useful_vars_available]}")
print(f"Advanced Metrics: {[v for v in advanced_metrics if v in useful_vars_available]}")

# Variables to exclude (less useful for FPL analysis)
excluded_vars = [var for var in numerical_cols if var not in useful_vars_available]
print(f"\n=== EXCLUDED VARIABLES ===")
print(f"Less useful for FPL: {excluded_vars}")

# Create filtered dataset with useful variables only
useful_numerical_df = df[useful_vars_available].copy()
print(f"\n=== FILTERED DATASET INFO ===")
print(f"Shape: {useful_numerical_df.shape}")
print(f"Useful numerical variables: {useful_vars_available}")

=== FILTERING USEFUL NUMERICAL VARIABLES ===
Original numerical variables: 31
Useful numerical variables: 26
Variables removed: 5

=== USEFUL VARIABLES BY CATEGORY ===
Core Performance: ['total_points', 'minutes', 'now_cost', 'selected_by_percent']
Attacking Metrics: ['G', 'A', 'xG', 'xA', 'shots', 'SoT', 'key_passes']
Expected Stats: ['xG', 'xA', 'xGI', 'npxG', 'npxGI', 'xP']
Defensive Metrics: ['CS', 'xCS', 'GC', 'xGC', 'tackles', 'recoveries', 'clearances_blocks_interceptions', 'defensive_contribution']
Advanced Metrics: ['PvsxP', 'carries_final_third', 'carries_penalty_area']

=== EXCLUDED VARIABLES ===
Less useful for FPL: ['id', 'element_type', 'gameweek', 'SiB', 'npG']

=== FILTERED DATASET INFO ===
Shape: (11013, 26)
Useful numerical variables: ['xP', 'CS', 'recoveries', 'npxGI', 'GC', 'xG', 'PvsxP', 'now_cost', 'clearances_blocks_interceptions', 'A', 'SoT', 'carries_final_third', 'xGI', 'key_passes', 'xCS', 'G', 'shots', 'total_points', 'selected_by_percent', 'tackles', 'carri

In [28]:
import pandas as pd
import warnings

# Define team short name mapping
team_short_names = {
    'Liverpool': 'LIV',
    'Man City': 'MCI',
    'Man Utd': 'MUN',
    'Chelsea': 'CHE',
    'Crystal Palace': 'CRY',
    'Bournemouth': 'BOU',
    'Spurs': 'TOT',
    'Everton': 'EVE',
    "Nott'm Forest": 'NFO',
    'Brighton': 'BHA',
    'Newcastle': 'NEW',
    'West Ham': 'WHU',
    'Sunderland': 'SUN',
    'Fulham': 'FUL',
    'Leeds': 'LEE',
    'Aston Villa': 'AVL',
    'Brentford': 'BRE',
    'Wolves': 'WOL',
    'Burnley': 'BUR'
}

# 🔧 FIXED: Standardized to 5 gameweeks to match team rankings calculation
FORM_GAMEWEEKS = 5  # Changed from 3 to 5 for consistency

def add_team_short_names(season_data: pd.DataFrame) -> pd.DataFrame:
    """
    Add team_name_short column to season_data based on team_name mapping.
    
    Args:
        season_data: DataFrame containing player season statistics
    Returns:
        Updated DataFrame with team_name_short column
    """
    # Create a copy to avoid modifying the original
    season_data = season_data.copy()
    
    # Print unique team names for diagnostics
    unique_teams = season_data['team_name'].unique()
    print("Unique team names in dataframe:", unique_teams)
    
    # Normalize team names for mapping (case-insensitive, strip spaces/punctuation)
    normalized_mapping = {k.lower().replace("'", "").strip(): v for k, v in team_short_names.items()}
    
    # Add team_name_short column with normalized matching
    def map_team_name(team_name):
        if pd.isna(team_name):
            return None
        normalized_name = team_name.lower().replace("'", "").strip()
        return normalized_mapping.get(normalized_name, team_name[:3].upper())  # Default to first 3 letters if unmapped
    
    season_data['team_name_short'] = season_data['team_name'].apply(map_team_name)
    
    # Check for unmapped team names (NaN or defaulted to first 3 letters)
    unmapped_teams = season_data[
        season_data['team_name_short'].isna() | 
        ~season_data['team_name'].str.lower().replace("'", "").str.strip().isin(normalized_mapping.keys())
    ]['team_name'].unique()
    if len(unmapped_teams) > 0:
        warnings.warn(f"Unmapped team names (assigned default short names): {unmapped_teams}. Consider updating the team_short_names mapping.")
    
    return season_data


# Apply the mapping
df = add_team_short_names(df)

Unique team names in dataframe: ['Arsenal' 'Aston Villa' 'Bournemouth' 'Brentford' 'Man Utd' 'Brighton'
 'Man City' 'Burnley' 'Chelsea' 'Everton' 'Sunderland' 'Crystal Palace'
 'Fulham' 'Leeds' 'Liverpool' 'Newcastle' "Nott'm Forest" 'Spurs'
 'West Ham' 'Wolves']


C:\Users\ITKOKM\AppData\Local\Temp\ipykernel_21636\3053468294.py:64: UserWarning: Unmapped team names (assigned default short names): ['Arsenal' "Nott'm Forest"]. Consider updating the team_short_names mapping.
  warnings.warn(f"Unmapped team names (assigned default short names): {unmapped_teams}. Consider updating the team_short_names mapping.")


In [29]:
# Display the first 20 rows of the dataset
print("=== TOP 20 ROWS OF DATASET ===")
print(df.head(20))


=== TOP 20 ROWS OF DATASET ===
    id  element_type      web_name team_name opponent_team_name  was_home  \
0    1             1          Raya   Arsenal            Man Utd     False   
1    2             1  Arrizabalaga   Arsenal            Man Utd     False   
2    3             1          Hein   Arsenal            Man Utd     False   
3    4             1       Setford   Arsenal            Man Utd     False   
4    5             2       Gabriel   Arsenal            Man Utd     False   
5    6             2        Saliba   Arsenal            Man Utd     False   
6    7             2     Calafiori   Arsenal            Man Utd     False   
7    8             2      J.Timber   Arsenal            Man Utd     False   
8    9             2        Kiwior   Arsenal            Man Utd     False   
9   10             2  Lewis-Skelly   Arsenal            Man Utd     False   
10  11             2         White   Arsenal            Man Utd     False   
11  12             2     Zinchenko   Arsenal 

In [30]:
# Outlier Detection and Analysis
print("=== OUTLIER DETECTION ===")

def detect_outliers_iqr(df, column):
    """Detect outliers using IQR method"""
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Analyze outliers for key metrics
key_metrics = ['total_points', 'now_cost', 'selected_by_percent', 'minutes']

for metric in key_metrics:
    if metric in df.columns and df[metric].notna().sum() > 0:
        outliers, lower, upper = detect_outliers_iqr(df, metric)
        print(f"\n{metric.upper()}:")
        print(f"  Normal range: {lower:.2f} to {upper:.2f}")
        print(f"  Number of outliers: {len(outliers)}")
        print(f"  Percentage of outliers: {(len(outliers) / len(df)) * 100:.2f}%")
        
        if len(outliers) > 0 and len(outliers) <= 10:
            print("  Top outliers:")
            top_outliers = outliers.nlargest(10, metric)[['web_name', 'team_name', metric]]
            for _, player in top_outliers.iterrows():
                print(f"    {player['web_name']} ({player['team_name']}): {player[metric]}")


=== OUTLIER DETECTION ===

TOTAL_POINTS:
  Normal range: -1.50 to 2.50
  Number of outliers: 1682
  Percentage of outliers: 15.27%

NOW_COST:
  Normal range: 2.80 to 6.80
  Number of outliers: 634
  Percentage of outliers: 5.76%

SELECTED_BY_PERCENT:
  Normal range: -1.25 to 2.35
  Number of outliers: 1757
  Percentage of outliers: 15.95%

MINUTES:
  Normal range: -103.50 to 172.50
  Number of outliers: 0
  Percentage of outliers: 0.00%


# 3️⃣ Exploratory Data Analysis {#exploratory-data-analysis}

## 🔍 Deep Dive into Data Patterns
Exploring data distributions, outliers, and relationships between variables.

In [31]:
# Positional and Team Analysis
print("=== POSITIONAL ANALYSIS ===")

# Position mapping
position_map = {1: 'Goalkeeper', 2: 'Defender', 3: 'Midfielder', 4: 'Forward'}
df['position_name'] = df['element_type'].map(position_map)

# Analysis by position
position_stats = df.groupby('position_name').agg({
    'total_points': ['count', 'mean', 'median', 'max'],
    'now_cost': ['mean', 'median'],
    'minutes': ['mean'],
    'selected_by_percent': ['mean'],
    'G': ['mean'],
    'A': ['mean']
}).round(2)

print("Position Statistics:")
print(position_stats)

print("\n=== TEAM ANALYSIS ===")

# Team performance analysis
team_stats = df.groupby('team_name').agg({
    'total_points': ['count', 'sum', 'mean'],
    'now_cost': ['mean'],
    'selected_by_percent': ['mean'],
    'G': ['sum'],
    'A': ['sum'],
    'minutes': ['sum']
}).round(2)

team_stats.columns = ['_'.join(col) for col in team_stats.columns]
team_stats = team_stats.sort_values('total_points_sum', ascending=False)

print("\nTop 10 Teams by Total Points:")
print(team_stats.head(10)[['total_points_sum', 'total_points_mean', 'now_cost_mean']])

print("\n=== VALUE ANALYSIS BY POSITION ===")
# Calculate points per million by position
df['points_per_million'] = df['total_points'] / df['now_cost']

value_by_position = df[df['total_points'] > 0].groupby('position_name')['points_per_million'].agg([
    'count', 'mean', 'median', 'max'
]).round(2)

print(value_by_position)



=== POSITIONAL ANALYSIS ===
Position Statistics:
              total_points                  now_cost        minutes  \
                     count  mean median max     mean median    mean   
position_name                                                         
Defender              3635  1.30    0.0  24     4.45    4.3   30.45   
Forward               1199  1.25    0.0  17     5.75    5.4   22.84   
Goalkeeper            1274  0.78    0.0  15     4.28    4.0   21.05   
Midfielder            4905  1.30    0.0  20     5.31    5.0   26.19   

              selected_by_percent     G     A  
                             mean  mean  mean  
position_name                                  
Defender                     2.06  0.02  0.02  
Forward                      3.68  0.10  0.02  
Goalkeeper                   2.32  0.00  0.00  
Midfielder                   1.51  0.05  0.04  

=== TEAM ANALYSIS ===

Top 10 Teams by Total Points:
                total_points_sum  total_points_mean  now_cost_m

# 5️⃣ Player Performance Analysis {#player-performance-analysis}

## 🏆 Feature 1 Season Leaders, Value Picks & Hidden Gems
Analysis of top performers using **cumulative season statistics** (not single gameweek data).

In [32]:
# Aggregate gameweek data to create season statistics for each player
print("="*60)
print("CREATING SEASON STATISTICS FROM GAMEWEEK DATA")
print("="*60)

# Group by player (id) and aggregate all gameweek data
season_stats = df.groupby(['id', 'web_name', 'team_name', 'team_name_short', 'position_name', 'element_type']).agg({
    # Core stats
    'total_points': 'sum',
    'now_cost': 'last',
    'selected_by_percent': 'last',
    'minutes': 'sum',
    'gameweek': ['max', 'count'],
    
    # Attacking stats
    'G': 'sum',
    'A': 'sum',
    'xG': 'sum',
    'xA': 'sum',
    'xGI': 'sum',
    'shots': 'sum',
    'SoT': 'sum',
    'SiB': 'sum',
    'key_passes': 'sum',
    
    # Defensive stats
    'CS': 'sum',
    'GC': 'sum',
    'xCS': 'sum',
    'xGC': 'sum',
    'tackles': 'sum',
    'recoveries': 'sum',
    'clearances_blocks_interceptions': 'sum',
    'defensive_contribution': 'sum'
}).reset_index()

# Flatten multi-level columns
season_stats.columns = ['_'.join(col).strip('_') if col[1] else col[0] for col in season_stats.columns]

# Rename aggregated columns to clean names
season_stats.rename(columns={
    'now_cost_last': 'now_cost',
    'selected_by_percent_last': 'selected_by_percent',
    'gameweek_max': 'last_gameweek',
    'gameweek_count': 'games_played',
    'total_points_sum': 'season_points',
    'minutes_sum': 'season_minutes',
    'G_sum': 'season_goals',
    'A_sum': 'season_assists',
    'xG_sum': 'season_xG',
    'xA_sum': 'season_xA',
    'xGI_sum': 'season_xGI',
    'shots_sum': 'season_shots',
    'SoT_sum': 'season_SoT',
    'SiB_sum': 'season_SiB',
    'key_passes_sum': 'season_key_passes',
    'CS_sum': 'season_CS',
    'GC_sum': 'season_GC',
    'xCS_sum': 'season_xCS',
    'xGC_sum': 'season_xGC',
    'tackles_sum': 'season_tackles',
    'recoveries_sum': 'season_recoveries',
    'clearances_blocks_interceptions_sum': 'clearances_blocks_interceptions_sum',
    'defensive_contribution_sum': 'defensive_contribution_sum'
}, inplace=True)

# Calculate per-game metrics
season_stats['points_per_game'] = season_stats['season_points'] / season_stats['games_played']
season_stats['goals_per_game'] = season_stats['season_goals'] / season_stats['games_played']
season_stats['assists_per_game'] = season_stats['season_assists'] / season_stats['games_played']
season_stats['minutes_per_game'] = season_stats['season_minutes'] / season_stats['games_played']

# Calculate points per million (needed for hidden gems)
season_stats['points_per_million'] = season_stats['season_points'] / season_stats['now_cost']

# Round numerical columns
numeric_cols = season_stats.select_dtypes(include=[np.number]).columns
season_stats[numeric_cols] = season_stats[numeric_cols].round(2)

print(f"Created season stats for {len(season_stats)} players")
print(f"Data covers gameweeks 1-{df['gameweek'].max()}")
season_stats.head(4)

CREATING SEASON STATISTICS FROM GAMEWEEK DATA
Created season stats for 775 players
Data covers gameweeks 1-15
Created season stats for 775 players
Data covers gameweeks 1-15


,id,web_name,team_name,team_name_short,position_name,element_type,season_points,now_cost,selected_by_percent,season_minutes,...,season_xGC,season_tackles,season_recoveries,clearances_blocks_interceptions_sum,defensive_contribution_sum,points_per_game,goals_per_game,assists_per_game,minutes_per_game,points_per_million
0,1,Raya,Arsenal,ARS,Goalkeeper,1,66,6.0,34.4,1350,...,8.6,0,118,11,0,4.4,0.0,0.0,90.0,11.0
1,2,Arrizabalaga,Arsenal,ARS,Goalkeeper,1,0,4.2,0.4,0,...,8.6,0,0,0,0,0.0,0.0,0.0,0.0,0.0
2,3,Hein,Arsenal,ARS,Goalkeeper,1,0,4.0,0.2,0,...,8.6,0,0,0,0,0.0,0.0,0.0,0.0,0.0
3,4,Setford,Arsenal,ARS,Goalkeeper,1,0,3.9,0.2,0,...,8.6,0,0,0,0,0.0,0.0,0.0,0.0,0.0


In [33]:
# Calculate the required metrics
num_players = season_stats['web_name'].nunique()
total_teams = season_stats['team_name'].nunique()
total_gameweeks = season_stats['last_gameweek'].max()

# Create a summary DataFrame
layout_df = pd.DataFrame({
    'number_of_players': [num_players],
    'total_teams': [total_teams],
    'total_gameweeks': [total_gameweeks]
})

print("Layout Data:")
print(layout_df)
# Export to JSON file
layout_df.to_json('backend/data/layout.json', orient='records', indent=4)

print("Layout data exported to backend/data/layout.json")

Layout Data:
   number_of_players  total_teams  total_gameweeks
0                738           20               15
Layout data exported to backend/data/layout.json


In [34]:
# Top Performers
print("🏆 === FPL KEY INSIGHTS & RECOMMENDATIONS ===")

# 🔧 FIXED: Calculate form for all players using last 5 gameweeks (was 3, now matches team rankings)
def calculate_player_form(player_name, team_name):
    """Calculate form as points per game from recent performances"""
    player_games = df[(df['web_name'] == player_name) & (df['team_name'] == team_name)]
    if len(player_games) == 0:
        print(f"Warning: No data for {player_name} ({team_name})")
        return None  # Changed from 5.0 to None for missing data
    
    # Get last 5 gameweeks (matches team rankings calculation)
    recent_games = player_games.nlargest(5, 'gameweek')
    if len(recent_games) == 0:
        print(f"Warning: No recent games for {player_name} ({team_name})")
        return None  # Changed from 5.0 to None
    
    avg_points = recent_games['total_points'].mean()
    element_type = player_games['element_type'].iloc[0] if 'element_type' in player_games else 3  # Default to MID if missing
    
    if element_type == 1:  # Goalkeeper
        form_score = min(10.0, max(0.0, avg_points * 1.2))
    elif element_type == 2:  # Defender
        form_score = min(10.0, max(0.0, avg_points * 1.1))
    else:  # Midfielder or Forward
        form_score = min(10.0, max(0.0, avg_points * 0.9))
    
    return round(form_score, 1)

# Add form to season_stats
season_stats['form'] = season_stats.apply(
    lambda row: calculate_player_form(row['web_name'], row['team_name']),
    axis=1
)

# Fill missing form with median to avoid NaN issues
if season_stats['form'].isna().any():
    median_form = season_stats['form'].median()
    season_stats['form'].fillna(median_form, inplace=True)
    print(f"⚠️ Filled {season_stats['form'].isna().sum()} missing form values with median: {median_form}")


# Now proceed with insights calculations
insights_data = {}

# 1. SEASON PERFORMERS - Top 15 by total points
print(f"\n🏆 TOP SEASON PERFORMERS")
print("-" * 50)

top_scorers = season_stats.nlargest(15, 'season_points')
top_scorer_names = set(top_scorers['web_name'].values)

season_performers_data = []
for i, (_, player) in enumerate(top_scorers.iterrows(), 1):
    ppg = player['season_points'] / player['games_played'] if player['games_played'] > 0 else 0
    
    player_data = {
        "player": player['web_name'],
        "team": player['team_name'],
        "team_short": player['team_name_short'],
        "position": player['position_name'],
        "points": int(player['season_points']),
        "ppg": round(ppg, 1),
        "price": player['now_cost'],
        "ownership": player['selected_by_percent'],
        "form": player['form']
    }
    season_performers_data.append(player_data)
    
    print(f"{i}. {player['web_name']} ({player['position_name']}, {player['team_name']} [{player['team_name_short']}])")
    print(f"   {player['season_points']:.0f} pts ({ppg:.1f} ppg) | £{player['now_cost']}m | {player['selected_by_percent']:.1f}% owned | Form: {player['form']}")

insights_data['season_performers'] = season_performers_data

# 2. VALUE PLAYERS - Best points per million (excluding top performers)
print(f"\n💰 BEST VALUE PLAYERS (Points per Million)")
print("-" * 50)

value_candidates = season_stats[
    (~season_stats['web_name'].isin(top_scorer_names)) & 
    (season_stats['season_points'] >= 15)
].copy()

value_candidates['points_per_million'] = value_candidates['season_points'] / value_candidates['now_cost']
value_players = value_candidates.nlargest(10, 'points_per_million')

value_players_data = []
for i, (_, player) in enumerate(value_players.iterrows(), 1):
    player_data = {
        "player": player['web_name'],
        "team": player['team_name'],
        "team_short": player['team_name_short'],
        "position": player['position_name'],
        "pointsPerMillion": round(player['points_per_million'], 2),
        "totalPoints": int(player['season_points']),
        "price": player['now_cost'],
        "form": player['form']
    }
    value_players_data.append(player_data)
    
    print(f"{i}. {player['web_name']} ({player['position_name']}, {player['team_name']} [{player['team_name_short']}])")
    print(f"   {player['points_per_million']:.2f} pts/£m | {player['season_points']:.0f} pts | £{player['now_cost']}m | Form: {player['form']}")

insights_data['value_players'] = value_players_data

# 3. HIDDEN GEMS - Low ownership with strong underlying stats
print(f"\n💎 HIDDEN GEMS (Low Ownership + Strong Potential)")
print("-" * 50)

# Compute dynamic thresholds based on averages
avg_points = season_stats['season_points'].mean()
avg_form = season_stats['form'].mean()
min_games = 4  # Minimum games played, reasonable for Gameweek 6
min_xG = season_stats['season_xG'].mean() * 0.8  # 80% of average xG for attacking threat

print(f"Dynamic thresholds: Avg Points = {avg_points:.2f}, Avg Form = {avg_form:.2f}, Min xG = {min_xG:.2f}")

# Filter hidden gems using dynamic thresholds
hidden_gems = season_stats[
    (season_stats['season_points'] >= avg_points * 0.8) &  # 80% of average points
    (season_stats['selected_by_percent'] < 8) &
    (season_stats['selected_by_percent'] > 0) &
    (season_stats['games_played'] >= min_games) &
    (season_stats['season_xG'] >= min_xG) &  # Dynamic xG threshold
    (season_stats['form'] >= avg_form * 0.8) &  # 80% of average form
    (~season_stats['web_name'].isin(top_scorer_names))
]

# Replace position names
hidden_gems['position_name'] = hidden_gems['position_name'].replace({
    'Forward': 'FWD',
    'Midfielder': 'MID',
    'Defender': 'DEF',
    'Goalkeeper': 'GK'
})

hidden_gems_data = []

if len(hidden_gems) > 0:
    hidden_gems = hidden_gems.copy()

    # Define metrics for z-score calculation
    metrics = [
        'season_xG', 'season_xA', 'season_xCS', 'season_key_passes',
        'form', 'points_per_game', 'goals_per_game', 'assists_per_game',
        'points_per_million', 'minutes_per_game',
        'season_tackles', 'season_recoveries', 'defensive_contribution_sum'
    ]

    # Calculate z-scores for each metric
    for metric in metrics:
        if metric in hidden_gems.columns:
            mean = hidden_gems[metric].mean()
            std = hidden_gems[metric].std()
            if std > 0:
                hidden_gems[f'{metric}_z'] = (hidden_gems[metric] - mean) / std
            else:
                hidden_gems[f'{metric}_z'] = 0

    # Calculate per-90 metrics (normalize by minutes played)
    hidden_gems['xG_per_game'] = hidden_gems['season_xG'] / (hidden_gems['season_minutes'] / 90)
    hidden_gems['xA_per_game'] = hidden_gems['season_xA'] / (hidden_gems['season_minutes'] / 90)

    # Calculate potential score based on position
    for idx, player in hidden_gems.iterrows():
        position = player['position_name']
        
        # Position-specific potential scores
        if position == 'FWD':
            potential_score = (
                player.get('season_xG_z', 0) * 0.30 +
                player.get('season_xA_z', 0) * 0.20 +
                player.get('form_z', 0) * 0.35 +
                player.get('points_per_game_z', 0) * 0.15 +
                player.get('points_per_million_z', 0) * 0.10
            )
        elif position == 'MID':
            potential_score = (
                player.get('season_xG_z', 0) * 0.25 +
                player.get('season_xA_z', 0) * 0.25 +
                player.get('form_z', 0) * 0.25 +
                player.get('season_key_passes_z', 0) * 0.15 +
                player.get('points_per_million_z', 0) * 0.10
            )
        elif position == 'DEF':
            potential_score = (
                player.get('season_xCS_z', 0) * 0.30 +
                player.get('season_xA_z', 0) * 0.15 +
                player.get('form_z', 0) * 0.25 +
                player.get('defensive_contribution_sum_z', 0) * 0.20 +
                player.get('points_per_million_z', 0) * 0.10
            )
        elif position == 'GK':
            potential_score = (
                player.get('season_xCS_z', 0) * 0.35 +
                player.get('form_z', 0) * 0.25 +
                player.get('points_per_game_z', 0) * 0.20 +
                player.get('points_per_million_z', 0) * 0.20
            )
        else:
            potential_score = 0

        hidden_gems.loc[idx, 'potential_score'] = potential_score

    # Normalize potential score to 0-10 scale
    if hidden_gems['potential_score'].std() > 0:
        min_score = hidden_gems['potential_score'].min()
        max_score = hidden_gems['potential_score'].max()
        hidden_gems['potential_score'] = ((hidden_gems['potential_score'] - min_score) / (max_score - min_score)) * 10
    
    # Sort by potential score
    hidden_gems_sorted = hidden_gems.nlargest(10, 'potential_score')
    
    for i, (_, player) in enumerate(hidden_gems_sorted.iterrows(), 1):
        player_data = {
            "player": player['web_name'],
            "team": player['team_name'],
            "team_short": player['team_name_short'],
            "position": player['position_name'],
            "points": int(player['season_points']),
            "ppg": round(player['points_per_game'], 2),
            "xG": round(player['season_xG'], 1),
            "xA": round(player['season_xA'], 1),
            "ownership": player['selected_by_percent'],
            "price": player['now_cost'],
            "form": player['form'],
            "potentialScore": round(player['potential_score'], 1)
        }
        hidden_gems_data.append(player_data)
        
        print(f"{i}. {player['web_name']} ({player['position_name']}, {player['team_name']} [{player['team_name_short']}])")
        print(f"   {player['season_points']:.0f} pts ({player['points_per_game']:.2f} ppg) | xG: {player['season_xG']:.1f}, xA: {player['season_xA']:.1f} | {player['selected_by_percent']:.1f}% owned | £{player['now_cost']}m")
        print(f"   Potential Score: {player['potential_score']:.1f}/10 | Form: {player['form']}")
else:
    print("No hidden gems found with current thresholds")

insights_data['hidden_gems'] = hidden_gems_data

# 4. GOAL LEADERS - Top goal scorers
print(f"\n⚽ TOP GOAL SCORERS")
print("-" * 50)

goal_leaders = season_stats[season_stats['season_goals'] > 0].nlargest(15, 'season_goals')

goal_scorers_data = []
for i, (_, player) in enumerate(goal_leaders.iterrows(), 1):
    gpg = player['season_goals'] / player['games_played'] if player['games_played'] > 0 else 0
    
    player_data = {
        "player": player['web_name'],
        "team": player['team_name'],
        "team_short": player['team_name_short'],
        "goals": int(player['season_goals']),
        "goalsPerGame": round(gpg, 2),
        "xG": round(player['season_xG'], 1),
        "points": int(player['season_points']),
        "price": player['now_cost'],
        "ownership": player['selected_by_percent'],
        "form": player['form']
    }
    goal_scorers_data.append(player_data)
    
    print(f"{i}. {player['web_name']} ({player['team_name']} [{player['team_name_short']}])")
    print(f"   {player['season_goals']:.0f} goals ({gpg:.2f} per game) | xG: {player['season_xG']:.1f} | {player['season_points']:.0f} pts | £{player['now_cost']}m | Form: {player['form']}")

insights_data['goal_scorers'] = goal_scorers_data

# 5. ASSIST LEADERS - Top assist providers
print(f"\n🎯 TOP ASSIST PROVIDERS")
print("-" * 50)

assist_leaders = season_stats[season_stats['season_assists'] > 0].nlargest(12, 'season_assists')

assist_providers_data = []
for i, (_, player) in enumerate(assist_leaders.iterrows(), 1):
    apg = player['season_assists'] / player['games_played'] if player['games_played'] > 0 else 0
    
    player_data = {
        "player": player['web_name'],
        "team": player['team_name'],
        "team_short": player['team_name_short'],
        "assists": int(player['season_assists']),
        "assistsPerGame": round(apg, 2),
        "points": int(player['season_points']),
        "price": player['now_cost'],
        "ownership": player['selected_by_percent'],
        "form": player['form']
    }
    assist_providers_data.append(player_data)
    
    print(f"{i}. {player['web_name']} ({player['team_name']} [{player['team_name_short']}])")
    print(f"   {player['season_assists']:.0f} assists ({apg:.2f} per game) | {player['season_points']:.0f} pts | £{player['now_cost']}m | Form: {player['form']}")

insights_data['assist_providers'] = assist_providers_data

# 6. DEFENSIVE LEADERS - Best defenders/goalkeepers
print(f"\n🛡️ DEFENSIVE LEADERS")
print("-" * 50)

defensive_candidates = season_stats[
    (season_stats['season_points'] >= 10) &
    (season_stats['games_played'] >= 3) &
    (season_stats['position_name'].isin(['Goalkeeper', 'Defender']))
].copy()

defensive_leaders_data = []
if len(defensive_candidates) > 0:
    defensive_candidates['defensive_score'] = (
        defensive_candidates['season_CS'] * 0.20 +  # Clean Sheets should be more heavily weighted
        defensive_candidates['season_tackles'] * 0.15 +  # Tackles are key for defensive performance
        defensive_candidates['season_recoveries'] * 0.15 +  # Recoveries are a crucial defensive stat
        defensive_candidates['season_xCS'] * 0.20 +  # xCS is predictive of future clean sheets, so it's very important
        defensive_candidates['defensive_contribution_sum'] * 0.10 +  # Overall defensive contributions
        defensive_candidates['clearances_blocks_interceptions_sum'] * 0.10 +  # Key to defensive stability
        (defensive_candidates['season_points'] / defensive_candidates['games_played']) * 0.10  # Points still matter, but not as much as the core defensive metrics
    )
    
    top_defenders = defensive_candidates.nlargest(10, 'defensive_score')
    print("🛡️ Best Defensive Performers:")
    for i, (_, player) in enumerate(top_defenders.iterrows(), 1):
        cs_rate = (player['season_CS'] / player['games_played']) * 100 if player['games_played'] > 0 else 0
        ppg = player['season_points'] / player['games_played'] if player['games_played'] > 0 else 0
        
        player_data = {
            "player": player['web_name'],
            "team": player['team_name'],
            "team_short": player['team_name_short'],
            "position": player['position_name'],
            "points": int(player['season_points']),
            "ppg": round(ppg, 1),
            "cleanSheets": int(player['season_CS']),
            "csRate": round(cs_rate, 1),
            "tackles": int(player['season_tackles']) if player['season_tackles'] > 0 else 1,
            "defensiveContributions": int(player['defensive_contribution_sum']) if 'defensive_contribution_sum' in player and player['defensive_contribution_sum'] > 0 else 1,
            "price": player['now_cost'],
            "form": player['form']
        }
        defensive_leaders_data.append(player_data)
        
        print(f"  {i}. {player['web_name']} ({player['team_name']} [{player['team_name_short']}], {player['position_name']})")
        print(f"     {player['season_points']:.0f} pts ({ppg:.1f} ppg) | {player['season_CS']:.0f} CS ({cs_rate:.1f}%) | {player['season_tackles']:.0f} tackles | £{player['now_cost']}m | Form: {player['form']}")

insights_data['defensive_leaders'] = defensive_leaders_data




# Export to JSON files
import json
import os

# Create output directory
output_dir = 'backend/data/top_performers'
os.makedirs(output_dir, exist_ok=True)

# Export each category to separate JSON files
for category, data in insights_data.items():
    filename = f'{output_dir}/{category}.json'
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
    print(f"\n✅ Exported {category}: {len(data)} players -> {filename}")

# Also create a combined file for convenience
combined_filename = f'{output_dir}/all_insights.json'
with open(combined_filename, 'w', encoding='utf-8') as f:
    json.dump(insights_data, f, indent=2, ensure_ascii=False)

print(f"\n🎉 ALL DATA EXPORTED TO JSON!")
print(f"📁 Output directory: {output_dir}/")
print(f"📊 Individual files: {list(insights_data.keys())}")
print(f"📦 Combined file: all_insights.json")

🏆 === FPL KEY INSIGHTS & RECOMMENDATIONS ===

🏆 TOP SEASON PERFORMERS
--------------------------------------------------
1. Haaland (Forward, Man City [MCI])
   122 pts (8.1 ppg) | £15.0m | 73.0% owned | Form: 4.3
2. Guéhi (Defender, Crystal Palace [CRY])
   91 pts (6.1 ppg) | £5.2m | 36.1% owned | Form: 5.9
3. Muñoz (Defender, Crystal Palace [CRY])
   89 pts (5.9 ppg) | £6.1m | 26.7% owned | Form: 7.7
4. Rice (Midfielder, Arsenal [ARS])
   84 pts (5.6 ppg) | £7.1m | 22.5% owned | Form: 3.8
5. Semenyo (Midfielder, Bournemouth [BOU])
   83 pts (5.5 ppg) | £7.6m | 46.1% owned | Form: 1.4
6. Chalobah (Defender, Chelsea [CHE])
   82 pts (5.5 ppg) | £5.3m | 11.9% owned | Form: 7.3
7. Gabriel (Defender, Arsenal [ARS])
   81 pts (5.4 ppg) | £6.2m | 15.9% owned | Form: 0.2
8. Bruno G. (Midfielder, Newcastle [NEW])
   81 pts (5.4 ppg) | £6.9m | 11.8% owned | Form: 6.3
9. Thiago (Forward, Brentford [BRE])
   80 pts (5.3 ppg) | £6.9m | 30.0% owned | Form: 5.9
10. Lacroix (Defender, Crystal Palace

C:\Users\ITKOKM\AppData\Local\Temp\ipykernel_21636\2857299135.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hidden_gems['position_name'] = hidden_gems['position_name'].replace({


In [35]:
# 7. PERFORMANCE ANALYSIS
print(f"\n📈 OVERPERFORMANCE ANALYSIS")
print("-" * 50)

overperformers_data = []
sustainable_scorers_data = []
underperformers_data = []

if 'season_xG' in season_stats.columns and 'season_goals' in season_stats.columns:
    overperformance_candidates = season_stats[
        (season_stats['season_goals'] > 0) & 
        (season_stats['season_xG'] > 0) &
        (season_stats['games_played'] >= 3)
    ].copy()
    
    # Calculate overperformance and normalize by minutes played
    overperformance_candidates['goal_overperformance'] = overperformance_candidates['season_goals'] - overperformance_candidates['season_xG']
    if 'minutes_played' in season_stats.columns:
        overperformance_candidates['overperformance_per_90'] = overperformance_candidates['goal_overperformance'] / overperformance_candidates['minutes_played'] * 90
    else:
        overperformance_candidates['overperformance_per_90'] = overperformance_candidates['goal_overperformance'] / overperformance_candidates['games_played']

    # Dynamic threshold based on xG
    overperformance_candidates['threshold'] = 0.1 * overperformance_candidates['season_xG'].clip(lower=0.5)  # Minimum threshold of 0.5

    # Goal overperformers (regression risk)
    goal_overperformers = overperformance_candidates[
        overperformance_candidates['goal_overperformance'] > overperformance_candidates['threshold']
    ].nlargest(8, 'overperformance_per_90')
    
    print("⚡ Top Goal Overperformers (Potential Regression Risk):")
    for i, (_, player) in enumerate(goal_overperformers.iterrows(), 1):
        player_data = {
            "player": player['web_name'],
            "team": player['team_name'],
            "team_short": player['team_name_short'],
            "goals": int(player['season_goals']),
            "xG": round(player['season_xG'], 1),
            "overperformance": round(player['goal_overperformance'], 1),
            "overperformance_per_90": round(player['overperformance_per_90'], 3),
            "sustainable": False,
            "form": player['form']
        }
        overperformers_data.append(player_data)
        shots_info = f" | Shots: {player['shots']:.0f}, SoT: {player['shots_on_target']:.0f}" if 'shots' in player and 'shots_on_target' in player else ""
        print(f"  {i}. {player['web_name']} ({player['team_name']} [{player['team_name_short']}]): {player['season_goals']:.0f} goals vs {player['season_xG']:.1f} xG (+{player['goal_overperformance']:.1f}) | Per 90: {player['overperformance_per_90']:.3f} | Form: {player['form']}{shots_info}")

    # Sustainable scorers (goals close to xG)
    sustainable_scorers = overperformance_candidates[
        abs(overperformance_candidates['goal_overperformance']) <= overperformance_candidates['threshold']
    ].nlargest(8, 'season_goals')  # Sort by goals for relevance
    print("\n🌟 Sustainable Scorers (Consistent Performance):")
    for i, (_, player) in enumerate(sustainable_scorers.iterrows(), 1):
        player_data = {
            "player": player['web_name'],
            "team": player['team_name'],
            "team_short": player['team_name_short'],
            "goals": int(player['season_goals']),
            "xG": round(player['season_xG'], 1),
            "overperformance": round(player['goal_overperformance'], 1),
            "overperformance_per_90": round(player['overperformance_per_90'], 3),
            "sustainable": True,
            "form": player['form']
        }
        sustainable_scorers_data.append(player_data)
        shots_info = f" | Shots: {player['shots']:.0f}, SoT: {player['shots_on_target']:.0f}" if 'shots' in player and 'shots_on_target' in player else ""
        print(f"  {i}. {player['web_name']} ({player['team_name']} [{player['team_name_short']}]): {player['season_goals']:.0f} goals vs {player['season_xG']:.1f} xG ({player['goal_overperformance']:.1f}) | Per 90: {player['overperformance_per_90']:.3f} | Form: {player['form']}{shots_info}")

    # Underperformers (potential breakout candidates)
    goal_underperformers = overperformance_candidates[
        overperformance_candidates['goal_overperformance'] < -overperformance_candidates['threshold']
    ].nlargest(8, 'season_xG')  # Sort by xG for breakout potential
    print("\n🔥 Goal Underperformers (Potential Breakout Candidates):")
    for i, (_, player) in enumerate(goal_underperformers.iterrows(), 1):
        player_data = {
            "player": player['web_name'],
            "team": player['team_name'],
            "team_short": player['team_name_short'],
            "goals": int(player['season_goals']),
            "xG": round(player['season_xG'], 1),
            "overperformance": round(player['goal_overperformance'], 1),
            "overperformance_per_90": round(player['overperformance_per_90'], 3),
            "sustainable": False,
            "form": player['form']
        }
        underperformers_data.append(player_data)
        shots_info = f" | Shots: {player['shots']:.0f}, SoT: {player['shots_on_target']:.0f}" if 'shots' in player and 'shots_on_target' in player else ""
        print(f"  {i}. {player['web_name']} ({player['team_name']} [{player['team_name_short']}]): {player['season_goals']:.0f} goals vs {player['season_xG']:.1f} xG ({player['goal_overperformance']:.1f}) | Per 90: {player['overperformance_per_90']:.3f} | Form: {player['form']}{shots_info}")

    # Export to JSON
    os.makedirs('backend/data/performance_analysis', exist_ok=True)
    with open('backend/data/performance_analysis/overperformers.json', 'w', encoding='utf-8') as f:
        json.dump(overperformers_data, f, indent=4, ensure_ascii=False) 
    with open('backend/data/performance_analysis/sustainable_scorers.json', 'w', encoding='utf-8') as f:
        json.dump(sustainable_scorers_data, f, indent=4 , ensure_ascii=False)
    with open('backend/data/performance_analysis/underperformers.json', 'w', encoding='utf-8') as f:
        json.dump(underperformers_data, f, indent=4 , ensure_ascii=False)
    print("\nExported performance data to backend/data/performance_analysis/")
else:
    print("❌ Missing required columns (season_xG or season_goals) for overperformance analysis")


📈 OVERPERFORMANCE ANALYSIS
--------------------------------------------------
⚡ Top Goal Overperformers (Potential Regression Risk):
  1. Bruno G. (Newcastle [NEW]): 5 goals vs 2.1 xG (+2.9) | Per 90: 0.193 | Form: 6.3
  2. Richarlison (Spurs [TOT]): 6 goals vs 3.2 xG (+2.8) | Per 90: 0.187 | Form: 3.8
  3. Gravenberch (Liverpool [LIV]): 3 goals vs 0.4 xG (+2.6) | Per 90: 0.173 | Form: 2.9
  4. Cash (Aston Villa [AVL]): 3 goals vs 0.6 xG (+2.4) | Per 90: 0.160 | Form: 5.9
  5. Buendía (Aston Villa [AVL]): 4 goals vs 1.6 xG (+2.4) | Per 90: 0.160 | Form: 3.6
  6. Caicedo (Chelsea [CHE]): 3 goals vs 0.6 xG (+2.4) | Per 90: 0.160 | Form: 0.0
  7. Dewsbury-Hall (Everton [EVE]): 4 goals vs 1.6 xG (+2.4) | Per 90: 0.160 | Form: 8.6
  8. Wilson (Fulham [FUL]): 4 goals vs 1.6 xG (+2.4) | Per 90: 0.160 | Form: 5.2

🌟 Sustainable Scorers (Consistent Performance):
  1. Woltemade (Newcastle [NEW]): 5 goals vs 4.8 xG (0.2) | Per 90: 0.017 | Form: 3.8
  2. João Pedro (Chelsea [CHE]): 4 goals vs 4.2

# 6️⃣ Strategic Analysis Tools {#strategic-analysis-tools}

## ⚔️ Advanced FPL Analysis Functions

This section contains powerful, reusable functions for Fantasy Premier League strategic analysis:

### 🔧 **Available Tools:**
1. **Defender Rankings** - Rank defenders by clean sheet potential and value
2. **Attacker Rankings** - Rank attacking players by goal/assist potential  
3. **Team Strength Analysis** - Calculate attacking and defensive strength for all teams
4. **Fixture Difficulty Calculator** - Score any specific matchup

### 📊 **Key Features:**
- Uses **cumulative season statistics** for accuracy
- Considers expected stats (xG, xA, xCS) for sustainability  
- Includes value scoring (points per £million)
- Accounts for consistency and minutes played
- Easily customizable parameters

# 7️⃣ Feature 2 Ranking Leaderboard


In [36]:
season_stats

,id,web_name,team_name,team_name_short,position_name,element_type,season_points,now_cost,selected_by_percent,season_minutes,...,season_tackles,season_recoveries,clearances_blocks_interceptions_sum,defensive_contribution_sum,points_per_game,goals_per_game,assists_per_game,minutes_per_game,points_per_million,form
0,1,Raya,Arsenal,ARS,Goalkeeper,1,66,6.0,34.4,1350,...,0,118,11,0,4.4,0.00,0.00,90.0,11.00,3.4
1,2,Arrizabalaga,Arsenal,ARS,Goalkeeper,1,0,4.2,0.4,0,...,0,0,0,0,0.0,0.00,0.00,0.0,0.00,0.0
2,3,Hein,Arsenal,ARS,Goalkeeper,1,0,4.0,0.2,0,...,0,0,0,0,0.0,0.00,0.00,0.0,0.00,0.0
3,4,Setford,Arsenal,ARS,Goalkeeper,1,0,3.9,0.2,0,...,0,0,0,0,0.0,0.00,0.00,0.0,0.00,0.0
4,5,Gabriel,Arsenal,ARS,Defender,2,81,6.2,15.9,990,...,12,15,88,100,5.4,0.07,0.13,66.0,13.06,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770,755,Lacey,Man Utd,MUN,Midfielder,3,0,4.5,0.0,0,...,0,0,0,0,0.0,0.00,0.00,0.0,0.00,0.0
771,756,DaCosta,Bournemouth,BOU,Midfielder,3,0,4.5,0.0,0,...,0,0,0,0,0.0,0.00,0.00,0.0,0.00,0.0
772,757,Hollingshead,Aston Villa,AVL,Goalkeeper,1,0,4.0,0.0,0,...,0,0,0,0,0.0,0.00,0.00,0.0,0.00,0.0
773,758,Mohamadou Kanté,West Ham,WHU,Midfielder,3,1,4.5,0.0,7,...,0,0,0,0,0.5,0.00,0.00,3.5,0.22,0.4


## 🏆 Step 1: Create Team Rankings with Form Weighting

This section creates comprehensive team strength rankings that blend:
- **60% Recent Form** (last 5 gameweeks) - captures current momentum
- **40% Season Average** - maintains stability

These rankings are used by subsequent cells for fixture analysis and player recommendations.

In [37]:
import json

# 🏆 ENHANCED TEAM STRENGTH RANKINGS WITH FORM WEIGHTING
print("="*70)
print("📊 COMPREHENSIVE TEAM STRENGTH RANKINGS")
print("="*70)
print("💡 Enhanced with recent form weighting (60% last 5 GWs, 40% season avg)")
print("💡 Includes all available defensive metrics for accurate fixture assessment")

def create_comprehensive_team_strength_rankings(season_data: pd.DataFrame, raw_df: pd.DataFrame = None) -> pd.DataFrame:
    """
    Create comprehensive team strength rankings using all available defensive metrics.
    Enhanced calculation includes tackles, recoveries, clearances, and expected stats.
    NOW WITH FORM WEIGHTING: Recent 5 gameweeks weighted 60%, season average 40%
    """
    
    # 📈 CALCULATE RECENT FORM (Last 5 gameweeks) if raw data available
    recent_form_stats = None
    if raw_df is not None:
        try:
            # Get the last gameweek number
            max_gw = raw_df['gameweek'].max()
            recent_gw_start = max(1, max_gw - 4)  # Last 5 gameweeks
            
            # Filter for recent gameweeks only
            recent_df = raw_df[raw_df['gameweek'] >= recent_gw_start].copy()
            
            # Calculate recent form stats (attacking)
            recent_form_stats = {
                'attack': recent_df.groupby('team_name').agg({
                    'G': 'sum',
                    'xG': 'sum',
                    'A': 'sum',
                    'xA': 'sum',
                    'shots': 'sum',
                    'key_passes': 'sum',
                    'gameweek': 'count'  # games in recent period
                }).rename(columns={'gameweek': 'recent_games'}),
                'defense': recent_df[recent_df['element_type'].isin([1, 2])].groupby('team_name').agg({
                    'CS': 'mean',
                    'xCS': 'mean',
                    'GC': 'mean',
                    'xGC': 'mean',
                    'tackles': 'mean',
                    'recoveries': 'mean',
                    'clearances_blocks_interceptions': 'mean',
                    'defensive_contribution': 'mean',
                    'gameweek': 'count'
                }).rename(columns={'gameweek': 'recent_games'})
            }
            
            print(f"✅ Recent form calculated from GW {recent_gw_start} to {max_gw} ({max_gw - recent_gw_start + 1} gameweeks)")
        except Exception as e:
            print(f"⚠️ Could not calculate recent form: {e}")
            recent_form_stats = None
    
    # ⚽ ATTACKING STRENGTH CALCULATION (Season averages)
    attacking_stats = season_data.groupby('team_name').agg({
        'season_goals': 'sum',
        'season_xG': 'sum', 
        'season_assists': 'sum',
        'season_xA': 'sum',
        'season_shots': 'sum',
        'season_SoT': 'sum',
        'season_key_passes': 'sum',
        'games_played': 'mean'
    }).round(3)
    
    # 🛡️ COMPREHENSIVE DEFENSIVE STRENGTH CALCULATION
    # Include all defensive players (GK + DEF)
    defensive_players = season_data[season_data['element_type'].isin([1, 2])]
    
    if len(defensive_players) == 0:
        print("⚠️ Warning: No defensive players found in dataset")
        defensive_stats = pd.DataFrame(index=attacking_stats.index)
        # Set default values for missing defensive data
        default_values = {
            'season_CS': 3.0, 'season_xCS': 3.0, 'season_GC': 1.5, 'season_xGC': 1.5,
            'season_tackles': 15.0, 'season_recoveries': 20.0, 
            'season_CBI': 10.0, 'season_defensive_contribution': 5.0,
            'games_played': attacking_stats['games_played'].iloc[0] if len(attacking_stats) > 0 else 6
        }
        for col, val in default_values.items():
            defensive_stats[col] = val
    else:
        # Aggregate all available defensive metrics
        agg_dict = {
            'games_played': 'mean'
        }
        
        # Add available defensive columns
        defensive_columns = ['season_CS', 'season_xCS', 'season_GC', 'season_xGC',
                           'season_tackles', 'season_recoveries', 
                           'season_clearances_blocks_interceptions', 'season_defensive_contribution']
        
        for col in defensive_columns:
            if col in defensive_players.columns:
                agg_dict[col] = 'mean'
        
        defensive_stats = defensive_players.groupby('team_name').agg(agg_dict).round(3)
        
        # Rename long column name for easier handling
        if 'season_clearances_blocks_interceptions' in defensive_stats.columns:
            defensive_stats.rename(columns={'season_clearances_blocks_interceptions': 'season_CBI'}, inplace=True)
    
    # 📊 CALCULATE PER-GAME METRICS
    
    # Attacking per-game metrics
    attacking_stats['goals_pg'] = attacking_stats['season_goals'] / attacking_stats['games_played']
    attacking_stats['xG_pg'] = attacking_stats['season_xG'] / attacking_stats['games_played']
    attacking_stats['assists_pg'] = attacking_stats['season_assists'] / attacking_stats['games_played']
    attacking_stats['xA_pg'] = attacking_stats['season_xA'] / attacking_stats['games_played']
    attacking_stats['shots_pg'] = attacking_stats['season_shots'] / attacking_stats['games_played']
    attacking_stats['key_passes_pg'] = attacking_stats['season_key_passes'] / attacking_stats['games_played']
    
    # Defensive per-game metrics
    defensive_stats['CS_rate'] = defensive_stats['season_CS'] / defensive_stats['games_played']
    defensive_stats['xCS_rate'] = defensive_stats['season_xCS'] / defensive_stats['games_played']
    defensive_stats['GC_pg'] = defensive_stats['season_GC'] / defensive_stats['games_played']
    defensive_stats['xGC_pg'] = defensive_stats['season_xGC'] / defensive_stats['games_played']
 
    
    if 'season_tackles' in defensive_stats.columns:
        defensive_stats['tackles_pg'] = defensive_stats['season_tackles'] / defensive_stats['games_played']
    if 'season_recoveries' in defensive_stats.columns:
        defensive_stats['recoveries_pg'] = defensive_stats['season_recoveries'] / defensive_stats['games_played']
    if 'season_CBI' in defensive_stats.columns:
        defensive_stats['CBI_pg'] = defensive_stats['season_CBI'] / defensive_stats['games_played']
    if 'season_defensive_contribution' in defensive_stats.columns:
        defensive_stats['def_contrib_pg'] = defensive_stats['season_defensive_contribution'] / defensive_stats['games_played']
    
    # 🎯 ENHANCED STRENGTH CALCULATIONS (Season baseline)
    
    # Attack Strength (weighted combination of multiple metrics)
    attacking_stats['attack_strength'] = (
        attacking_stats['xG_pg'] * 0.25 +           # Expected goals (predictive)
        attacking_stats['goals_pg'] * 0.20 +        # Actual goals (results)
        attacking_stats['xA_pg'] * 0.20 +           # Expected assists (creativity)
        attacking_stats['assists_pg'] * 0.15 +      # Actual assists
        attacking_stats['shots_pg'] * 0.10 +        # Shot volume
        attacking_stats['key_passes_pg'] * 0.10     # Key passes (creativity)
    )
    
    # Comprehensive Defense Strength (using all available metrics)
    defense_components = []
    weights = []
    
    # Core defensive metrics (always available)
    defense_components.append(defensive_stats['CS_rate'])
    weights.append(0.25)  # Clean sheet rate
    
    defense_components.append(1 / (defensive_stats['GC_pg'] + 0.1))
    weights.append(0.20)  # Goals conceded (inverted)
    
    # Expected metrics (if available)
    if 'xCS_rate' in defensive_stats.columns:
        defense_components.append(defensive_stats['xCS_rate'])
        weights.append(0.15)  # Expected clean sheet rate
    
    if 'xGC_pg' in defensive_stats.columns:
        defense_components.append(1 / (defensive_stats['xGC_pg'] + 0.1))
        weights.append(0.15)  # Expected goals conceded (inverted)
    
    # 🎯 IMPROVED: Defensive actions with min-max normalization (more balanced)
    if 'tackles_pg' in defensive_stats.columns:
        tackles_norm = defensive_stats['tackles_pg'] / defensive_stats['tackles_pg'].max() if defensive_stats['tackles_pg'].max() > 0 else defensive_stats['tackles_pg']
        defense_components.append(tackles_norm)
        weights.append(0.10)
    
    if 'recoveries_pg' in defensive_stats.columns:
        recoveries_norm = defensive_stats['recoveries_pg'] / defensive_stats['recoveries_pg'].max() if defensive_stats['recoveries_pg'].max() > 0 else defensive_stats['recoveries_pg']
        defense_components.append(recoveries_norm)
        weights.append(0.05)
    
    if 'CBI_pg' in defensive_stats.columns:
        cbi_norm = defensive_stats['CBI_pg'] / defensive_stats['CBI_pg'].max() if defensive_stats['CBI_pg'].max() > 0 else defensive_stats['CBI_pg']
        defense_components.append(cbi_norm)
        weights.append(0.05)
    
    if 'def_contrib_pg' in defensive_stats.columns:
        def_contrib_norm = defensive_stats['def_contrib_pg'] / defensive_stats['def_contrib_pg'].max() if defensive_stats['def_contrib_pg'].max() > 0 else defensive_stats['def_contrib_pg']
        defense_components.append(def_contrib_norm)
        weights.append(0.05)
    
    # Normalize weights to sum to 1
    total_weight = sum(weights)
    weights = [w/total_weight for w in weights]
    
    # Calculate weighted defensive strength
    defensive_stats['defense_strength'] = sum(comp * weight for comp, weight in zip(defense_components, weights))
    
    # 🔥 BLEND RECENT FORM WITH SEASON AVERAGES (60% recent, 40% season)
    if recent_form_stats is not None:
        print("\n🔄 Blending recent form (60%) with season averages (40%)...")
        
        # Blend attacking strength
        for team in attacking_stats.index:
            if team in recent_form_stats['attack'].index:
                recent_attack = recent_form_stats['attack'].loc[team]
                recent_games = recent_attack['recent_games']
                
                if recent_games > 0:
                    # Calculate recent form strength using same weights
                    recent_attack_strength = (
                        (recent_attack['xG'] / recent_games) * 0.25 +
                        (recent_attack['G'] / recent_games) * 0.20 +
                        (recent_attack['xA'] / recent_games) * 0.20 +
                        (recent_attack['A'] / recent_games) * 0.15 +
                        (recent_attack['shots'] / recent_games) * 0.10 +
                        (recent_attack['key_passes'] / recent_games) * 0.10
                    )
                    
                    # Blend: 60% recent form, 40% season average
                    season_strength = attacking_stats.loc[team, 'attack_strength']
                    attacking_stats.loc[team, 'attack_strength'] = (
                        recent_attack_strength * 0.60 + season_strength * 0.40
                    )
        
        # Blend defensive strength
        for team in defensive_stats.index:
            if team in recent_form_stats['defense'].index:
                recent_defense = recent_form_stats['defense'].loc[team]
                recent_games = recent_defense['recent_games']
                
                if recent_games > 0:
                    # Calculate recent form defensive strength
                    recent_defense_components = []
                    recent_weights = []
                    
                    # Core metrics
                    recent_defense_components.append(recent_defense['CS'])
                    recent_weights.append(0.25)
                    
                    recent_defense_components.append(1 / (recent_defense['GC'] + 0.1))
                    recent_weights.append(0.20)
                    
                    if 'xCS' in recent_defense.index:
                        recent_defense_components.append(recent_defense['xCS'])
                        recent_weights.append(0.15)
                    
                    if 'xGC' in recent_defense.index:
                        recent_defense_components.append(1 / (recent_defense['xGC'] + 0.1))
                        recent_weights.append(0.15)
                    
                    # Normalize recent defensive actions
                    recent_def_df = recent_form_stats['defense']
                    if 'tackles' in recent_defense.index and recent_def_df['tackles'].max() > 0:
                        recent_defense_components.append(recent_defense['tackles'] / recent_def_df['tackles'].max())
                        recent_weights.append(0.10)
                    
                    if 'recoveries' in recent_defense.index and recent_def_df['recoveries'].max() > 0:
                        recent_defense_components.append(recent_defense['recoveries'] / recent_def_df['recoveries'].max())
                        recent_weights.append(0.05)
                    
                    if 'clearances_blocks_interceptions' in recent_defense.index and recent_def_df['clearances_blocks_interceptions'].max() > 0:
                        recent_defense_components.append(recent_defense['clearances_blocks_interceptions'] / recent_def_df['clearances_blocks_interceptions'].max())
                        recent_weights.append(0.05)
                    
                    if 'defensive_contribution' in recent_defense.index and recent_def_df['defensive_contribution'].max() > 0:
                        recent_defense_components.append(recent_defense['defensive_contribution'] / recent_def_df['defensive_contribution'].max())
                        recent_weights.append(0.05)
                    
                    # Normalize weights
                    total_recent_weight = sum(recent_weights)
                    recent_weights = [w/total_recent_weight for w in recent_weights]
                    
                    recent_defense_strength = sum(comp * weight for comp, weight in zip(recent_defense_components, recent_weights))
                    
                    # Blend: 60% recent form, 40% season average
                    season_strength = defensive_stats.loc[team, 'defense_strength']
                    defensive_stats.loc[team, 'defense_strength'] = (
                        recent_defense_strength * 0.60 + season_strength * 0.40
                    )
        
        print("✅ Form blending complete - Rankings now reflect recent performance!")
    else:
        print("ℹ️ Using season-long averages only (no recent form data)")
    
    # 🏆 COMBINE TEAM RANKINGS
    team_rankings = attacking_stats[['attack_strength']].join(
        defensive_stats[['defense_strength']], how='outer'
    )
    
    # 🔧 FIXED: Handle missing data (pandas 3.0 compatible)
    team_rankings = team_rankings.fillna({
        'attack_strength': team_rankings['attack_strength'].median(),
        'defense_strength': team_rankings['defense_strength'].median()
    })
    
    # Overall strength calculation
    team_rankings['overall_strength'] = (
        team_rankings['attack_strength'] * 0.6 + 
        team_rankings['defense_strength'] * 0.4
    )
    
    # Generate rankings
    team_rankings['attack_rank'] = team_rankings['attack_strength'].rank(ascending=False, method='dense').astype(int)
    team_rankings['defense_rank'] = team_rankings['defense_strength'].rank(ascending=False, method='dense').astype(int)
    team_rankings['overall_rank'] = team_rankings['overall_strength'].rank(ascending=False, method='dense').astype(int)
    
    return team_rankings.round(3)

# Generate comprehensive team rankings WITH FORM WEIGHTING
# Pass the raw df to enable form calculation
team_rankings = create_comprehensive_team_strength_rankings(season_stats, raw_df=df)
team_rankings_sorted = team_rankings.sort_values('overall_rank')

print("\n🏆 COMPREHENSIVE TEAM STRENGTH RANKINGS")
print("=" * 65)
print("📋 All Teams Ranked (Enhanced with Recent Form + Defensive Analysis):")
print(team_rankings_sorted[['overall_rank', 'attack_rank', 'defense_rank', 
                           'overall_strength', 'attack_strength', 'defense_strength']].to_string())

print(f"\n⚽ TOP ATTACKING TEAMS:")
attack_rankings = team_rankings.sort_values('attack_rank').head(20)
for idx, (team, data) in enumerate(attack_rankings.iterrows(), 1):
    team_short = season_stats[season_stats['team_name'] == team]['team_name_short'].iloc[0] if not season_stats[season_stats['team_name'] == team].empty else 'UNK'
    print(f" {int(data['attack_rank']):2d}. {team:<15} [{team_short}] (Attack: {data['attack_strength']:.3f})")

print(f"\n🛡️ TOP DEFENSIVE TEAMS:")
defense_rankings = team_rankings.sort_values('defense_rank').head(20)
for idx, (team, data) in enumerate(defense_rankings.iterrows(), 1):
    team_short = season_stats[season_stats['team_name'] == team]['team_name_short'].iloc[0] if not season_stats[season_stats['team_name'] == team].empty else 'UNK'
    print(f" {int(data['defense_rank']):2d}. {team:<15} [{team_short}] (Defense: {data['defense_strength']:.3f})")

📊 COMPREHENSIVE TEAM STRENGTH RANKINGS
💡 Enhanced with recent form weighting (60% last 5 GWs, 40% season avg)
💡 Includes all available defensive metrics for accurate fixture assessment
✅ Recent form calculated from GW 11 to 15 (5 gameweeks)

🔄 Blending recent form (60%) with season averages (40%)...
✅ Form blending complete - Rankings now reflect recent performance!

🏆 COMPREHENSIVE TEAM STRENGTH RANKINGS
📋 All Teams Ranked (Enhanced with Recent Form + Defensive Analysis):
                overall_rank  attack_rank  defense_rank  overall_strength  attack_strength  defense_strength
team_name                                                                                                   
Man City                   1            1             5             1.288            1.793             0.532
Arsenal                    2            3             2             1.287            1.664             0.722
Chelsea                    3            5             3             1.200            1

## 🏟️ Step 2: Calculate Dynamic Home Advantage

Now that we have team rankings, we calculate home/away performance splits for each team.
This provides team-specific home advantage factors instead of a static boost for all teams.

In [40]:
import pandas as pd
import numpy as np

# 🏟️ CALCULATE DYNAMIC HOME ADVANTAGE FROM ACTUAL DATA
print("="*70)
print("🏟️ CALCULATING DYNAMIC HOME ADVANTAGE (FIXED)")
print("="*70)

def calculate_home_away_advantage(raw_df, team_rankings):
    """
    Calculate actual home/away performance for each team from gameweek data.
    Returns home advantage factors for adjusting fixture difficulty dynamically.
    
    FIXED: Now counts unique FIXTURES (gameweeks), not player rows!
    
    Process:
    1. Separate home and away games for each team
    2. Calculate attack/defense strength for each context
    3. Compute advantage factor as (home_strength - away_strength) / away_strength
    4. Convert to rank adjustment (negative = better home performance)
    """
    
    home_away_advantage = {}
    
    for team in team_rankings.index:
        # Get all games for this team
        team_home = raw_df[(raw_df['team_name'] == team) & (raw_df['was_home'] == True)]
        team_away = raw_df[(raw_df['team_name'] == team) & (raw_df['was_home'] == False)]
        
        # 🔧 FIX: Count unique gameweeks (fixtures), not player rows
        home_fixtures = team_home['gameweek'].nunique()
        away_fixtures = team_away['gameweek'].nunique()
        
        if home_fixtures < 2 or away_fixtures < 2:
            # Not enough data - use moderate default
            home_away_advantage[team] = {
                'home_games': home_fixtures,
                'away_games': away_fixtures,
                'home_attack_str': 0.0,
                'away_attack_str': 0.0,
                'home_defense_str': 0.0,
                'away_defense_str': 0.0,
                'attack_advantage_factor': 0.0,  # No advantage if insufficient data
                'defense_advantage_factor': 0.0,
                'attack_rank_boost': 0.0,
                'defense_rank_boost': 0.0,
                'data_quality': 'insufficient'
            }
            continue
        
        # ⚽ CALCULATE HOME ATTACKING STRENGTH
        # Sum stats THEN divide by fixtures (not player rows)
        home_attack_strength = (
            (team_home['xG'].sum() / home_fixtures) * 0.25 +
            (team_home['G'].sum() / home_fixtures) * 0.20 +
            (team_home['xA'].sum() / home_fixtures) * 0.20 +
            (team_home['A'].sum() / home_fixtures) * 0.15 +
            (team_home['shots'].sum() / home_fixtures) * 0.10 +
            (team_home['key_passes'].sum() / home_fixtures) * 0.10
        )
        
        # ⚽ CALCULATE AWAY ATTACKING STRENGTH
        away_attack_strength = (
            (team_away['xG'].sum() / away_fixtures) * 0.25 +
            (team_away['G'].sum() / away_fixtures) * 0.20 +
            (team_away['xA'].sum() / away_fixtures) * 0.20 +
            (team_away['A'].sum() / away_fixtures) * 0.15 +
            (team_away['shots'].sum() / away_fixtures) * 0.10 +
            (team_away['key_passes'].sum() / away_fixtures) * 0.10
        )
        
        # 🛡️ CALCULATE HOME DEFENSIVE STRENGTH (for defenders/GK only)
        home_defenders = team_home[team_home['element_type'].isin([1, 2])]
        if len(home_defenders) > 0:
            # Use MAX for CS/GC per gameweek (team only gets 1 CS per game, not sum of all defenders)
            home_cs = home_defenders.groupby('gameweek')['CS'].max().sum()
            home_gc = home_defenders.groupby('gameweek')['GC'].max().sum()
            
            # 🔧 FIXED: Group by gameweek first to get team totals, then calculate average per fixture
            home_tackles_per_fixture = home_defenders.groupby('gameweek')['tackles'].sum().mean()
            home_recoveries_per_fixture = home_defenders.groupby('gameweek')['recoveries'].sum().mean()
            home_xCS_per_fixture = home_defenders.groupby('gameweek')['xCS'].sum().mean()
            
            home_defense_strength = (
                (home_cs / home_fixtures) * 0.35 +
                (1 / (home_gc / home_fixtures + 0.1)) * 0.30 +
                home_xCS_per_fixture * 0.20 +
                home_tackles_per_fixture * 0.10 +
                home_recoveries_per_fixture * 0.05
            )
        else:
            home_defense_strength = 0.0
        
        # 🛡️ CALCULATE AWAY DEFENSIVE STRENGTH
        away_defenders = team_away[team_away['element_type'].isin([1, 2])]
        if len(away_defenders) > 0:
            away_cs = away_defenders.groupby('gameweek')['CS'].max().sum()
            away_gc = away_defenders.groupby('gameweek')['GC'].max().sum()
            
            # 🔧 FIXED: Group by gameweek first to get team totals, then calculate average per fixture
            away_tackles_per_fixture = away_defenders.groupby('gameweek')['tackles'].sum().mean()
            away_recoveries_per_fixture = away_defenders.groupby('gameweek')['recoveries'].sum().mean()
            away_xCS_per_fixture = away_defenders.groupby('gameweek')['xCS'].sum().mean()
            
            away_defense_strength = (
                (away_cs / away_fixtures) * 0.35 +
                (1 / (away_gc / away_fixtures + 0.1)) * 0.30 +
                away_xCS_per_fixture * 0.20 +
                away_tackles_per_fixture * 0.10 +
                away_recoveries_per_fixture * 0.05
            )
        else:
            away_defense_strength = 0.0
        
        # 📊 CALCULATE ADVANTAGE FACTORS
        # Positive = better at home, Negative = better away
        if away_attack_strength > 0:
            attack_advantage_factor = (home_attack_strength - away_attack_strength) / away_attack_strength
        else:
            attack_advantage_factor = 0.0
        
        if away_defense_strength > 0:
            defense_advantage_factor = (home_defense_strength - away_defense_strength) / away_defense_strength
        else:
            defense_advantage_factor = 0.0
        
        # 🎯 CONVERT ADVANTAGE FACTOR TO RANK BOOST
        # More aggressive scaling (was 0.5, now 0.8) to better reflect home advantage
        
        total_teams = len(team_rankings)
        attack_rank_boost = attack_advantage_factor * (total_teams / 10) * 0.8
        defense_rank_boost = defense_advantage_factor * (total_teams / 10) * 0.8
        
        home_away_advantage[team] = {
            'home_games': home_fixtures,
            'away_games': away_fixtures,
            'home_attack_str': round(home_attack_strength, 3),
            'away_attack_str': round(away_attack_strength, 3),
            'home_defense_str': round(home_defense_strength, 3),
            'away_defense_str': round(away_defense_strength, 3),
            'attack_advantage_factor': round(attack_advantage_factor, 3),  # % difference
            'defense_advantage_factor': round(defense_advantage_factor, 3),
            'attack_rank_boost': round(attack_rank_boost, 2),  # Rank positions
            'defense_rank_boost': round(defense_rank_boost, 2),
            'data_quality': 'good' if home_fixtures >= 5 and away_fixtures >= 5 else 'limited'
        }
    
    return pd.DataFrame(home_away_advantage).T

# Generate home/away advantage data
home_away_df = calculate_home_away_advantage(df, team_rankings)

# Sort by attack advantage (most impactful home teams first)
home_away_sorted = home_away_df.sort_values('attack_advantage_factor', ascending=False)

print("\n🏆 HOME ADVANTAGE BY TEAM (FIXED)")
print("="*100)
print("\n📊 Top 10 Home Advantage Teams (Attack):")
print("Team                    | Fixtures (H/A) | Home Att | Away Att | Advantage | Rank Boost")
print("-" * 100)

for team, data in home_away_sorted.head(10).iterrows():
    print(f"{team:<23} | {int(data['home_games']):2d} / {int(data['away_games']):2d}       | "
          f"{data['home_attack_str']:7.3f}  | {data['away_attack_str']:7.3f} | "
          f"{data['attack_advantage_factor']:+7.1%} | {data['attack_rank_boost']:+5.2f} ranks")

print("\n🛡️ Top 10 Home Advantage Teams (Defense):")
defense_sorted = home_away_df.sort_values('defense_advantage_factor', ascending=False)
print("Team                    | Fixtures (H/A) | Home Def | Away Def | Advantage | Rank Boost")
print("-" * 100)

for team, data in defense_sorted.head(10).iterrows():
    print(f"{team:<23} | {int(data['home_games']):2d} / {int(data['away_games']):2d}       | "
          f"{data['home_defense_str']:7.3f}  | {data['away_defense_str']:7.3f} | "
          f"{data['defense_advantage_factor']:+7.1%} | {data['defense_rank_boost']:+5.2f} ranks")



print("\n📈 Data Quality Summary:")
print("✅ Fixed: Now uses unique fixtures instead of player rows!")

print(f"Teams with good data (5+ fixtures each): {len(home_away_df[home_away_df['data_quality'] == 'good'])}")
print("\n✅ Home/away advantage data calculated and ready for fixture analysis!")

print(f"Teams with limited data: {len(home_away_df[home_away_df['data_quality'] == 'limited'])}")# Save for use in fixture analyzer

print(f"Teams with insufficient data: {len(home_away_df[home_away_df['data_quality'] == 'insufficient'])}")

🏟️ CALCULATING DYNAMIC HOME ADVANTAGE (FIXED)

🏆 HOME ADVANTAGE BY TEAM (FIXED)

📊 Top 10 Home Advantage Teams (Attack):
Team                    | Fixtures (H/A) | Home Att | Away Att | Advantage | Rank Boost
----------------------------------------------------------------------------------------------------
Leeds                   |  8 /  7       |   3.594  |   1.919 |  +87.2% | +1.40 ranks
Nott'm Forest           |  7 /  8       |   3.572  |   2.083 |  +71.5% | +1.14 ranks
Wolves                  |  7 /  7       |   2.476  |   1.509 |  +64.2% | +1.03 ranks
Sunderland              |  7 /  8       |   2.835  |   1.751 |  +61.9% | +0.99 ranks
Everton                 |  8 /  7       |   3.244  |   2.071 |  +56.6% | +0.91 ranks
Fulham                  |  8 /  7       |   3.257  |   2.182 |  +49.3% | +0.79 ranks
Brentford               |  7 /  8       |   3.105  |   2.137 |  +45.3% | +0.72 ranks
Man Utd                 |  7 /  7       |   4.404  |   3.057 |  +44.1% | +0.71 ranks
Newcastle 

In [41]:
# Initialize lists to avoid duplicates
attacking_picks = []
defensive_picks = []

def get_players_for_matchup(team, matchup_type, season_stats, team_rankings, n=4):
    team_players = season_stats[season_stats['team_name'] == team].copy()
    if team_players.empty:
        return pd.DataFrame()
    
    # Set defaults for missing columns
    default_cols = {
        'season_xG': 0.0, 'season_xGC': 0.0, 'season_CS': 0.0, 'season_xCS': 0.0,
        'season_points': 0.0, 'season_goals': 0.0, 'season_assists': 0.0,
        'season_xA': 0.0, 'season_shots': 0.0, 'season_SoT': 0.0, 'season_SiB': 0.0,
        'season_minutes': 0.0, 'now_cost': 5.0, 'selected_by_percent': 0.0, 'form': 0.0
    }
    for col, val in default_cols.items():
        if col not in team_players.columns:
            team_players[col] = val
    
    # Filter out players with insufficient minutes (less than 180 minutes = 2 full games)
    # This prevents inflated per-90 stats for rarely-used substitutes
    min_minutes_threshold = 180
    team_players = team_players[team_players['season_minutes'] >= min_minutes_threshold]
    
    if team_players.empty:
        return pd.DataFrame()
    
    # Use minutes played / 90 instead of games_played for accurate per-game metrics
    team_players['games_equivalent'] = team_players['season_minutes'] / 90
    
    # Compute metrics using games_equivalent (minutes/90)
    team_players['points_per_game'] = team_players['season_points'] / team_players['games_equivalent']
    team_players['points_per_million'] = team_players['season_points'] / team_players['now_cost'].replace(0, 1)
    
    # 🔧 FIXED: Calculate consistency as avg minutes per appearance (rewards full 90min starters)
    team_players['consistency_score'] = np.minimum(
        (team_players['season_minutes'] / team_players['games_played']) / 90, 
        1
    )
    
    if matchup_type == 'weak_defense':
        team_players['xg_per_game'] = team_players['season_xG'] / team_players['games_equivalent']
        team_players['xa_per_game'] = team_players['season_xA'] / team_players['games_equivalent']
        team_players['goals_per_game'] = team_players['season_goals'] / team_players['games_equivalent']
        team_players['assists_per_game'] = team_players['season_assists'] / team_players['games_equivalent']
        team_players['shots_per_game'] = team_players['season_shots'] / team_players['games_equivalent']
        team_players['SoT_per_game'] = team_players['season_SoT'] / team_players['games_equivalent']
        team_players['SiB_per_game'] = team_players['season_SiB'] / team_players['games_equivalent']
        position_filter = team_players['position_name'].isin(['Forward', 'Midfielder'])
        # Composite attacker score
        team_players['attacker_score'] = (
            0.3 * team_players['xg_per_game'] +
            0.25 * team_players['xa_per_game'] +
            0.2 * team_players['goals_per_game'] +
            0.15 * team_players['assists_per_game'] +
            0.05 * team_players['SoT_per_game'] +
            0.05 * team_players['SiB_per_game']
        ) * 0.6 + 0.25 * team_players['points_per_million'] + 0.15 * team_players['consistency_score']
        sort_columns = ['attacker_score', 'points_per_game', 'xg_per_game']
        display_cols = [
            'web_name', 'position_name', 'now_cost', 'goals_per_game', 'assists_per_game',
            'xg_per_game', 'xa_per_game', 'shots_per_game', 'SoT_per_game', 'SiB_per_game',
            'points_per_game', 'points_per_million', 'consistency_score', 'selected_by_percent',
            'team_name_short', 'form', 'attacker_score'
        ]
    elif matchup_type == 'weak_attack':
        team_players['clean_sheet_rate'] = team_players['season_CS'] / team_players['games_equivalent']
        team_players['xcs_per_game'] = team_players['season_xCS'] / team_players['games_equivalent']
        team_players['xgc_per_game'] = team_players['season_xGC'] / team_players['games_equivalent']
        team_players['goals_conceded_per_game'] = team_players['season_GC'] / team_players['games_equivalent']
        position_filter = team_players['position_name'].isin(['Defender', 'Goalkeeper'])
        # Composite defender score
        team_players['defender_score'] = (
            0.4 * team_players['xcs_per_game'] +
            0.35 * team_players['clean_sheet_rate'] +
            0.15 / (team_players['goals_conceded_per_game'] + 0.1)
        ) * 0.6 + 0.25 * team_players['points_per_million'] + 0.15 * team_players['consistency_score']
        sort_columns = ['defender_score', 'clean_sheet_rate']
        display_cols = [
            'web_name', 'position_name', 'now_cost', 'clean_sheet_rate', 'xcs_per_game',
            'goals_conceded_per_game', 'points_per_game', 'points_per_million',
            'consistency_score', 'selected_by_percent', 'team_name_short', 'form', 'defender_score'
        ]
    else:
        return pd.DataFrame()
    
    filtered_players = team_players[position_filter]
    if filtered_players.empty:
        return pd.DataFrame()
    
    for col in sort_columns:
        if col not in filtered_players.columns:
            filtered_players[col] = 0.0
    
    result = filtered_players.sort_values(by=sort_columns, ascending=False).head(n)[display_cols]
    return result.round(3)

# SHOW ALL TEAMS: Complete attacking rankings with player recommendations
print(f"\n⚽ ATTACKING PICKS FROM ALL TEAMS (Sorted by Attack Rank):")
print("=" * 60)
all_attacking_teams = team_rankings.sort_values('attack_rank').head(20)  # Limit to top 20 teams

for idx, (team, data) in enumerate(all_attacking_teams.iterrows()):
    if team in season_stats['team_name'].values:
        attack_rank = int(data['attack_rank'])
        attack_strength = data['attack_strength']
        overall_strength = data['overall_strength']
        
        attackers = get_players_for_matchup(team, 'weak_defense', season_stats, team_rankings, 4)
        if not attackers.empty:
            print(f"\n🔴 {team} (#{attack_rank} Attack, Strength: {attack_strength:.3f}, Overall: {overall_strength:.3f}):")
            print(attackers.to_string(index=False))
            
            # Collect for JSON
            team_data = {
                'team': team,
                'attack_rank': attack_rank,
                'attack_strength': attack_strength,
                'overall_strength': overall_strength,
                'players': attackers.to_dict(orient='records')
            }
            attacking_picks.append(team_data)
        else:
            print(f"\n🔴 {team} (#{attack_rank} Attack, Strength: {attack_strength:.3f}, Overall: {overall_strength:.3f}): No attacking players found")

# SHOW ALL TEAMS: Complete defensive rankings with player recommendations  
print(f"\n🛡️ DEFENSIVE PICKS FROM ALL TEAMS (Sorted by Defense Rank):")
print("=" * 60)

all_defensive_teams = team_rankings.sort_values('defense_rank').head(20)  # Limit to top 20 teams

for idx, (team, data) in enumerate(all_defensive_teams.iterrows()):
    if team in season_stats['team_name'].values:
        defense_rank = int(data['defense_rank'])
        defense_strength = data['defense_strength']
        overall_strength = data['overall_strength']
        
        defenders = get_players_for_matchup(team, 'weak_attack', season_stats, team_rankings, 4)
        if not defenders.empty:
            print(f"\n🔵 {team} (#{defense_rank} Defense, Strength: {defense_strength:.3f}, Overall: {overall_strength:.3f}):")
            print(defenders.to_string(index=False))
            
            # Collect for JSON
            team_data = {
                'team': team,
                'defense_rank': defense_rank,
                'defense_strength': defense_strength,
                'overall_strength': overall_strength,
                'players': defenders.to_dict(orient='records')
            }
            defensive_picks.append(team_data)
        else:
            print(f"\n🔵 {team} (#{defense_rank} Defense, Strength: {defense_strength:.3f}, Overall: {overall_strength:.3f}): No defensive players found")

# Debugging: Print number of teams
print(f"\nProcessed {len(attacking_picks)} attacking teams")
print(f"Processed {len(defensive_picks)} defensive teams")

# Export to JSON
os.makedirs('backend/data/quick_picks', exist_ok=True)

with open('backend/data/quick_picks/attackingpicks.json', 'w', encoding='utf-8') as f:
    json.dump(attacking_picks, f, indent=4, ensure_ascii=False)

with open('backend/data/quick_picks/defensivepicks.json', 'w', encoding='utf-8') as f:

    json.dump(defensive_picks, f, indent=4, ensure_ascii=False)

print("Exported defensive picks to backend/data/quick_picks/defensivepicks.json")
print("\nExported attacking picks to backend/data/quick_picks/attackingpicks.json")


⚽ ATTACKING PICKS FROM ALL TEAMS (Sorted by Attack Rank):

🔴 Man City (#1 Attack, Strength: 1.793, Overall: 1.288):
 web_name position_name  now_cost  goals_per_game  assists_per_game  xg_per_game  xa_per_game  shots_per_game  SoT_per_game  SiB_per_game  points_per_game  points_per_million  consistency_score  selected_by_percent team_name_short  form  attacker_score
  Haaland       Forward      15.0           1.053             0.211        0.927        0.154           4.002         2.176         3.861            8.565               8.133              0.950                 73.0             MCI   4.3           2.692
    Foden    Midfielder       8.5           0.517             0.172        0.336        0.164           2.498         0.775         1.636            6.632               9.059              0.774                 24.2             MCI   8.8           2.616
     Doku    Midfielder       6.6           0.096             0.385        0.144        0.491           1.444         0.770 

attack_strength = (
    0.25 * xG_pg +
    0.20 * goals_pg +
    0.20 * xA_pg +
    0.15 * assists_pg +
    0.10 * shots_pg +
    0.10 * key_passes_pg
)

CS_rate	0.25	More clean sheets = better defense

1 / (GC_pg + 0.1)	0.20	Fewer goals conceded = stronger defense

xCS_rate (optional)	0.15	Model-based estimate of clean sheets

1 / (xGC_pg + 0.1)	0.15	Expected goals conceded (lower is better)

tackles_pg	0.10	Normalized by max in dataset

recoveries_pg	0.05	Normalized

CBI_pg	0.05	Normalized

def_contrib_pg	0.05	Normalized



attack_rank	Rank attack_strength, descending	1 = best attacking team
defense_rank	Rank defense_strength, descending	1 = best defensive team
overall_rank	Rank overall_strength, descending	1 = strongest all-around team

# 🔮  FIXTURE ANALYZER - SEASON-WIDE ANALYSIS



In [42]:
team_rankings

,attack_strength,defense_strength,overall_strength,attack_rank,defense_rank,overall_rank
team_name,,,,,,
Arsenal,1.664,0.722,1.287,3,2,2
Aston Villa,1.352,0.503,1.012,9,8,8
Bournemouth,1.367,0.478,1.012,8,10,9
Brentford,1.122,0.431,0.845,16,15,16
Brighton,1.450,0.463,1.055,6,11,7
Burnley,0.844,0.324,0.636,20,20,20
Chelsea,1.546,0.681,1.200,5,3,3
Crystal Palace,1.237,0.800,1.062,10,1,6
Everton,1.142,0.670,0.953,14,4,11


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json
import os

class EnhancedFixtureAnalyzer:
    """
    Advanced fixture analysis system for complete season planning
    
    Features:
    - Season-wide fixture difficulty analysis with realistic football scaling
    - Separate scaling for attacking vs defensive opportunities  
    - Visual heatmaps and charts
    - Strategic transfer timing recommendations
    - Position-specific insights
    - Team matchup intelligence
    """
    
    def __init__(self, season_stats, team_rankings, fixtures_path='fixture_template.csv'):
        """Initialize with your existing data"""
        self.season_stats = season_stats
        self.team_rankings = team_rankings
        self.fixtures_df = pd.read_csv(fixtures_path)
        self.current_gw = season_stats['last_gameweek'].max()
        self.start_gw = self.current_gw + 1
        self._process_data()
        
    def _process_data(self):
        """Process the data and create team mappings"""
        self._map_team_names()
    
    def _map_team_names(self):
        """Map fixture team names to season_stats team names"""
        fixture_teams = set(self.fixtures_df['home_team'].unique()) | set(self.fixtures_df['away_team'].unique())
        season_teams = set(self.season_stats['team_name'].unique())
        
        self.team_mapping = {}
        
        for fixture_team in fixture_teams:
            if fixture_team in season_teams:
                self.team_mapping[fixture_team] = fixture_team
                continue
            best_match = None
            for season_team in season_teams:
                if (fixture_team.lower().replace(' ', '') in season_team.lower().replace(' ', '') or
                    season_team.lower().replace(' ', '') in fixture_team.lower().replace(' ', '')):
                    best_match = season_team
                    break
            if best_match:
                self.team_mapping[fixture_team] = best_match
            else:
                self.team_mapping[fixture_team] = fixture_team
                print(f"⚠️ Could not match '{fixture_team}' - using default mapping")
                
    def get_fixture_difficulty_matrix(self, start_gw=None, end_gw=None, home_advantage=2):
        """Create fixture difficulty matrix using FOOTBALL-REALISTIC calculation method"""
        if start_gw is None:
            start_gw = self.fixtures_df['gameweek'].min()
        if end_gw is None:
            end_gw = self.fixtures_df['gameweek'].max()
            
        fixtures_period = self.fixtures_df[
            (self.fixtures_df['gameweek'] >= start_gw) & 
            (self.fixtures_df['gameweek'] <= end_gw)
        ].copy()
        
        difficulties = []
        total_teams = len(self.team_rankings)
        
        for _, fixture in fixtures_period.iterrows():
            home_team = self.team_mapping.get(fixture['home_team'], fixture['home_team'])
            away_team = self.team_mapping.get(fixture['away_team'], fixture['away_team'])
            
            if home_team in self.team_rankings.index and away_team in self.team_rankings.index:
                home_stats = self.team_rankings.loc[home_team]
                away_stats = self.team_rankings.loc[away_team]
                
                home_attack_rank = int(home_stats['attack_rank'])
                away_defense_rank = int(away_stats['defense_rank'])
                
                original_home_attack = home_attack_rank
                if home_advantage > 0 and home_attack_rank > 1:
                    home_attack_rank = max(1, home_attack_rank - home_advantage)
                
                attack_rank_difference = away_defense_rank - home_attack_rank
                attack_difficulty = attack_rank_difference / total_teams * 10
                
                home_defense_rank = int(home_stats['defense_rank'])
                away_attack_rank = int(away_stats['attack_rank'])
                
                original_home_defense = home_defense_rank
                if home_advantage > 0 and home_defense_rank > 1:
                    home_defense_rank = max(1, home_defense_rank - home_advantage)
                
                defense_rank_difference = away_attack_rank - home_defense_rank
                defense_difficulty = defense_rank_difference / total_teams * 10
                
                difficulties.append({
                    'gameweek': fixture['gameweek'],
                    'home_team': fixture['home_team'],
                    'away_team': fixture['away_team'],
                    'mapped_home': home_team,
                    'mapped_away': away_team,
                    'attack_difficulty': attack_difficulty,
                    'defense_difficulty': defense_difficulty,
                    'overall_difficulty': (attack_difficulty + defense_difficulty) / 2,
                    'home_attack_rank': home_attack_rank,
                    'away_defense_rank': away_defense_rank,
                    'home_defense_rank': home_defense_rank,
                    'away_attack_rank': away_attack_rank,
                    'attack_rank_diff': attack_rank_difference,
                    'defense_rank_diff': defense_rank_difference
                })
        
        return pd.DataFrame(difficulties)


    def export_fixture_data(self, num_gameweeks=6):
        """Export fixture data, opportunities, and team summaries to JSON for front-end"""
        
        def score_to_attacking_probability(difficulty_score):
            """
            Convert difficulty score to ATTACKING fixture rating with football-realistic scaling
            
            Attacking opportunities are more common in football:
            - Even weaker teams can score against stronger defenses
            - Top attackers can find opportunities against most defenses
            - Rating scale focuses on the 40-80% range for most realistic matchups
            
            Examples:
            Score +10 (best matchup) → ~85% rating
            Score +5 → ~75% rating  
            Score 0 (neutral) → ~60% rating
            Score -5 → ~45% rating
            Score -10 (worst matchup) → ~35% rating
            """
            # More generous scaling for attacking - reflects that goals are achievable
            scaled_score = difficulty_score * 0.25 + 1.2  # Shift curve upward
            probability = 100 / (1 + np.exp(-scaled_score))
            return round(max(25, min(90, probability)), 1)  # Cap between 25-90%
        
        def score_to_defensive_probability(difficulty_score):
            """
            Convert difficulty score to DEFENSIVE fixture rating with football-realistic scaling
            
            Clean sheets are much rarer in football:
            - Even the best defenses struggle against top attacks
            - Lower-rated defenses rarely keep clean sheets vs good attacks
            - Rating scale is more conservative, focusing on 15-65% range
            
            Examples:
            Score +10 (best matchup) → ~70% rating
            Score +5 → ~55% rating
            Score 0 (neutral) → ~35% rating  
            Score -5 → ~22% rating
            Score -10 (worst matchup) → ~15% rating
            """
            # More conservative scaling for defense - reflects clean sheet difficulty
            scaled_score = difficulty_score * 0.35 - 0.8  # Shift curve downward
            probability = 100 / (1 + np.exp(-scaled_score))
            return round(max(10, min(75, probability)), 1)  # Cap between 10-75%
        
        # Helper function to get team_name_short
        def get_team_short(team):
            mapped_team = self.team_mapping.get(team, team)
            team_data = self.season_stats[self.season_stats['team_name'] == mapped_team]
            return team_data['team_name_short'].iloc[0] if 'team_name_short' in team_data.columns and not team_data.empty else team
        
        # 1. Fixtures Data (all game weeks)
        start_gw = self.start_gw
        end_gw = start_gw
        difficulty_matrix = self.get_fixture_difficulty_matrix()  # All game weeks
        fixtures_data = []
        for _, fixture in difficulty_matrix.iterrows():
            home_team = fixture['home_team']
            away_team = fixture['away_team']
            gw = int(fixture['gameweek'])
            mapped_home = fixture['mapped_home']
            mapped_away = fixture['mapped_away']
            
            home_att_score = round(fixture['attack_difficulty'], 1)
            home_def_score = round(fixture['defense_difficulty'], 1)
            
            # Calculate away team scores
            away_att_score = 0.0
            away_def_score = 0.0
            if mapped_away in self.team_rankings.index and mapped_home in self.team_rankings.index:
                away_stats = self.team_rankings.loc[mapped_away]
                home_stats = self.team_rankings.loc[mapped_home]
                total_teams = len(self.team_rankings)
                
                away_attack_rank = int(away_stats['attack_rank'])
                home_defense_rank = int(home_stats['defense_rank'])
                
                # 🔧 FIXED: Apply home advantage to home team's defense (boost by 2 ranks)
                original_home_defense = home_defense_rank
                if home_defense_rank > 1:
                    home_defense_rank = max(1, home_defense_rank - 2)
                
                away_att_rank_diff = home_defense_rank - away_attack_rank
                away_att_score = round(away_att_rank_diff / total_teams * 10, 1)
                
                away_defense_rank = int(away_stats['defense_rank'])
                home_attack_rank = int(home_stats['attack_rank'])
                away_def_rank_diff = home_attack_rank - away_defense_rank
                away_def_score = round(away_def_rank_diff / total_teams * 10, 1)
            
            # Calculate fixture ratings using FOOTBALL-REALISTIC scaling
            home_attack_rating = score_to_attacking_probability(home_att_score)
            home_defense_rating = score_to_defensive_probability(home_def_score)
            away_attack_rating = score_to_attacking_probability(away_att_score)
            away_defense_rating = score_to_defensive_probability(away_def_score)
            
            fixture_data = {
                'gameweek': gw,
                'fixture': f"{home_team} vs {away_team}",
                'home_team': {
                    'name': home_team,
                    'short_name': get_team_short(home_team),
                    'attacking_fixture_rating': home_attack_rating,
                    'defensive_fixture_rating': home_defense_rating,
                    'rank': {
                        'attack': int(self.team_rankings.loc[mapped_home, 'attack_rank']) if mapped_home in self.team_rankings.index else 0,
                        'defense': int(self.team_rankings.loc[mapped_home, 'defense_rank']) if mapped_home in self.team_rankings.index else 0
                    }
                },
                'away_team': {
                    'name': away_team,
                    'short_name': get_team_short(away_team),
                    'attacking_fixture_rating': away_attack_rating,
                    'defensive_fixture_rating': away_defense_rating,
                    'rank': {
                        'attack': int(self.team_rankings.loc[mapped_away, 'attack_rank']) if mapped_away in self.team_rankings.index else 0,
                        'defense': int(self.team_rankings.loc[mapped_away, 'defense_rank']) if mapped_away in self.team_rankings.index else 0
                    }
                }
            }
            fixtures_data.append(fixture_data)
        
        # 2. Fixture Opportunities
        opportunities_data = {'attack': [], 'defense': []}

        # Filter fixtures for the next num_gameweeks
        start_gw = self.start_gw
        end_gw = start_gw + num_gameweeks - 1  # Cover exactly num_gameweeks
        relevant_fixtures = [f for f in fixtures_data if start_gw <= f['gameweek'] <= end_gw]

        if not relevant_fixtures:
            print(f"⚠️ No fixtures found for gameweeks {start_gw} to {end_gw}")
        else:
            for position_type in ['attack', 'defense']:
                all_opportunities = []
                
                for fixture in relevant_fixtures:
                    gw = fixture['gameweek']
                    home_team = fixture['home_team']['name']
                    away_team = fixture['away_team']['name']
                    
                    # Home team opportunity
                    all_opportunities.append({
                        'gameweek': gw,
                        'team': home_team,
                        'short_name': fixture['home_team']['short_name'],
                        'opponent': away_team,
                        'attacking_fixture_rating': fixture['home_team']['attacking_fixture_rating'],
                        'defensive_fixture_rating': fixture['home_team']['defensive_fixture_rating'],
                        'combined_score': round((fixture['home_team']['attacking_fixture_rating'] + fixture['home_team']['defensive_fixture_rating']) / 2, 1),
                        'venue': 'H'
                    })
                    
                    # Away team opportunity
                    all_opportunities.append({
                        'gameweek': gw,
                        'team': away_team,
                        'short_name': fixture['away_team']['short_name'],
                        'opponent': home_team,
                        'attacking_fixture_rating': fixture['away_team']['attacking_fixture_rating'],
                        'defensive_fixture_rating': fixture['away_team']['defensive_fixture_rating'],
                        'combined_score': round((fixture['away_team']['attacking_fixture_rating'] + fixture['away_team']['defensive_fixture_rating']) / 2, 1),
                        'venue': 'A'
                    })

                # Sort by the appropriate rating for each position type
                if position_type == 'attack':
                    all_opportunities.sort(key=lambda x: x['attacking_fixture_rating'], reverse=True)
                else:  # defense
                    all_opportunities.sort(key=lambda x: x['defensive_fixture_rating'], reverse=True)
                    
                opportunities_data[position_type] = all_opportunities[:10]

        # 3. Team Fixture Summary (all game weeks) - ENHANCED WITH FIXTURE SWING ANALYSIS
        all_difficulties = self.get_fixture_difficulty_matrix(start_gw, end_gw)
        if all_difficulties.empty:
            print("❌ No fixture difficulty data available")
            return []

        team_summary = []
        fixture_teams = set(all_difficulties['home_team'].unique()) | set(all_difficulties['away_team'].unique())

        for team in fixture_teams:
            # Filter all fixtures for the team (home and away)
            team_fixtures = all_difficulties[(all_difficulties['home_team'] == team) | (all_difficulties['away_team'] == team)]
            if len(team_fixtures) == 0:
                continue

            attack_scores = []
            defense_scores = []
            favorable_fixtures = 0
            favorable_home_fixtures = 0
            
            # NEW: Track ratings by period for swing analysis
            near_term_ratings = []  # Next 3 GWs
            medium_term_ratings = []  # Following 3 GWs
            
            # NEW: Track home fixtures by period
            near_term_home_count = 0
            medium_term_home_count = 0
            
            # Define period boundaries
            near_term_end = start_gw + 2  # First 3 gameweeks
            medium_term_end = start_gw + 5  # Following 3 gameweeks

            # Process each fixture
            for _, fixture in team_fixtures.iterrows():
                is_home = fixture['home_team'] == team
                current_gw = fixture['gameweek']
                
                if is_home:
                    # Home fixture: use precomputed difficulties
                    attack_diff = fixture['attack_difficulty']
                    defense_diff = fixture['defense_difficulty']
                    # For rating: convert difficulty score to rating using football-realistic scaling
                    attack_rating = score_to_attacking_probability(attack_diff)
                    defense_rating = score_to_defensive_probability(defense_diff)
                else:
                    # Away fixture: recalculate from away team's perspective
                    mapped_away = fixture['mapped_away']
                    mapped_home = fixture['mapped_home']
                    
                    if mapped_away in self.team_rankings.index and mapped_home in self.team_rankings.index:
                        away_stats = self.team_rankings.loc[mapped_away]
                        home_stats = self.team_rankings.loc[mapped_home]
                        total_teams = len(self.team_rankings)
                        
                        # Away team attack difficulty
                        away_attack_rank = int(away_stats['attack_rank'])
                        home_defense_rank = int(home_stats['defense_rank'])
                        away_att_rank_diff = home_defense_rank - away_attack_rank
                        attack_diff = away_att_rank_diff / total_teams * 10
                        
                        # Away team defense difficulty
                        away_defense_rank = int(away_stats['defense_rank'])
                        home_attack_rank = int(home_stats['attack_rank'])
                        away_def_rank_diff = home_attack_rank - away_defense_rank
                        defense_diff = away_def_rank_diff / total_teams * 10
                        
                        # Convert to ratings
                        attack_rating = score_to_attacking_probability(attack_diff)
                        defense_rating = score_to_defensive_probability(defense_diff)
                    else:
                        continue

                attack_scores.append(attack_diff)
                defense_scores.append(defense_diff)
                
                # NEW: Track ratings for swing analysis
                avg_fixture_rating = (attack_rating + defense_rating) / 2
                if current_gw <= near_term_end:
                    near_term_ratings.append(avg_fixture_rating)
                    if is_home:
                        near_term_home_count += 1
                elif current_gw <= medium_term_end:
                    medium_term_ratings.append(avg_fixture_rating)
                    if is_home:
                        medium_term_home_count += 1

                # Count favorable fixtures (and track if they're home fixtures)
                if attack_diff >= 2.5 or defense_diff >= 2.5:
                    favorable_fixtures += 1
                    if is_home:
                        favorable_home_fixtures += 1

            if not attack_scores or not defense_scores:
                continue

            # Compute averages
            avg_attack_diff = round(np.mean(attack_scores), 3)
            avg_defense_diff = round(np.mean(defense_scores), 3)
            overall_diff = round((avg_attack_diff + avg_defense_diff) / 2, 3)
            
            # NEW: Calculate fixture swing metrics
            near_term_rating = round(np.mean(near_term_ratings), 1) if near_term_ratings else 50.0
            medium_term_rating = round(np.mean(medium_term_ratings), 1) if medium_term_ratings else 50.0
            fixture_swing = round(medium_term_rating - near_term_rating, 1)
            
            # Categorize swing and provide form context
            if fixture_swing >= 10:
                swing_category = "Fixture Improvement"
                form_context = "buy_opportunity"  # Getting easier - good time to buy
                swing_emoji = "📈"
            elif fixture_swing <= -10:
                swing_category = "Fixture Decline"
                form_context = "sell_warning"  # Getting harder - consider selling
                swing_emoji = "📉"
            else:
                swing_category = "Stable"
                form_context = "hold"
                swing_emoji = "➡️"

            team_summary.append({
                'team': team,
                'avg_attack_difficulty': avg_attack_diff,
                'avg_defense_difficulty': avg_defense_diff,
                'overall_difficulty': overall_diff,
                'num_favorable_fixtures': favorable_fixtures,
                # NEW: Period-specific home fixture counts
                'near_term_home_fixtures': near_term_home_count,
                'medium_term_home_fixtures': medium_term_home_count,
                # NEW: Fixture swing analysis fields
                'near_term_rating': near_term_rating,  # Next 3 GWs
                'medium_term_rating': medium_term_rating,  # Following 3 GWs
                'fixture_swing': fixture_swing,
                'swing_category': swing_category,
                'swing_emoji': swing_emoji,
                'form_context': form_context
            })
        
        summary_data = sorted(team_summary, key=lambda x: x['overall_difficulty'], reverse=True)
        # Save to JSON
        os.makedirs('backend/data/fixture_analysis', exist_ok=True)

        with open('backend/data/fixture_analysis/fixtures.json', 'w') as f:
            json.dump(fixtures_data, f, indent=4)
        with open('backend/data/fixture_analysis/fixture_opportunities.json', 'w') as f:
            json.dump(opportunities_data, f, indent=4)
        with open('backend/data/fixture_analysis/team_fixture_summary.json', 'w') as f:
            json.dump(summary_data, f, indent=4)

        print("\nExported fixture data to backend/data/fixture_analysis/fixtures.json")
        print("Exported fixture opportunities to backend/data/fixture_analysis/fixture_opportunities.json")
        print("Exported team fixture summary to backend/data/fixture_analysis/team_fixture_summary.json")

# Initialization block
print("🔮 INITIALIZING ENHANCED FIXTURE ANALYZER...")
print("=" * 60)

try:
    analyzer = EnhancedFixtureAnalyzer(season_stats, team_rankings, 'fixture_template.csv')
    print("✅ Analyzer initialized successfully!")
    print(f"📊 Fixture data loaded: {len(analyzer.fixtures_df)} fixtures")
    print(f"📅 Gameweeks available: {analyzer.fixtures_df['gameweek'].min()} to {analyzer.fixtures_df['gameweek'].max()}")
    print(f"🏟️ Teams mapped: {len(analyzer.team_mapping)} teams")
    
    missing_mappings = [team for team, mapped in analyzer.team_mapping.items() 
                       if mapped not in analyzer.team_rankings.index and mapped == team]
    
    if missing_mappings:
        print(f"⚠️ Teams without ranking data: {', '.join(missing_mappings[:5])}")
        print("   (These teams will be skipped in analysis)")
    else:
        print("✅ All teams successfully mapped to ranking data")
    
    print("\n🎯 ENHANCED FIXTURE ANALYZER READY!")
    
except Exception as e:
    print(f"❌ Error initializing analyzer: {e}")
    print("Please check that 'fixture_template.csv' exists and has the correct format")
    import traceback
    traceback.print_exc()

# Export fixture data to JSON
if 'analyzer' in locals():
    print("\n" + "="*70)
    print("📤 EXPORTING FIXTURE DATA TO JSON")
    print("="*70)
    analyzer.export_fixture_data()  # Export results to JSON
    print("✅ Fixture data exported successfully!")
else:
    print("❌ Cannot export - analyzer was not initialized successfully")

SyntaxError: invalid syntax (1762771598.py, line 479)

In [ ]:
# Export fixture data to JSON
print("\n" + "="*70)
print("📤 EXPORTING FIXTURE DATA TO JSON")
print("="*70)

if 'analyzer' in locals():
    try:
        analyzer.export_fixture_data(num_gameweeks=6)  # Export results to JSON
        print("✅ Fixture data exported successfully!")
    except Exception as e:
        print(f"❌ Error during export: {e}")
        import traceback
        traceback.print_exc()
else:
    print("❌ Cannot export - analyzer was not initialized successfully")

## ✅ Feature Validation & Flow Summary

This section validates that all advanced features are working correctly and integrated properly.

In [ ]:
import os
import json

print("="*80)
print("✅ FEATURE VALIDATION & FLOW CHECK")
print("="*80)

# Check 1: Form-Weighted Rankings
print("\n1️⃣ FORM-WEIGHTED TEAM RANKINGS")
print("-" * 80)
if 'team_rankings' in locals():
    print(f"✅ Team rankings available: {len(team_rankings)} teams")
    print(f"   Columns: {', '.join(team_rankings.columns.tolist())}")
    print(f"   Attack Rank Range: {team_rankings['attack_rank'].min()}-{team_rankings['attack_rank'].max()}")
    print(f"   Defense Rank Range: {team_rankings['defense_rank'].min()}-{team_rankings['defense_rank'].max()}")
else:
    print("❌ Team rankings not found")

# Check 2: Dynamic Home Advantage
print("\n2️⃣ DYNAMIC HOME ADVANTAGE")
print("-" * 80)
if 'home_away_df' in locals():
    print(f"✅ Home/away advantage data available: {len(home_away_df)} teams")
    
    # Convert to numeric if needed
    df_numeric = home_away_df.copy()
    df_numeric['attack_advantage_factor'] = pd.to_numeric(df_numeric['attack_advantage_factor'], errors='coerce')
    df_numeric['defense_advantage_factor'] = pd.to_numeric(df_numeric['defense_advantage_factor'], errors='coerce')
    
    atk_min = df_numeric['attack_advantage_factor'].min()
    atk_max = df_numeric['attack_advantage_factor'].max()
    def_min = df_numeric['defense_advantage_factor'].min()
    def_max = df_numeric['defense_advantage_factor'].max()
    
    print(f"   Attack Advantage Range: {atk_min:+.1%} to {atk_max:+.1%}")
    print(f"   Defense Advantage Range: {def_min:+.1%} to {def_max:+.1%}")
    
    # Show top home teams
    top_home = df_numeric.nlargest(3, 'attack_advantage_factor')
    print(f"\n   🏆 Teams with Best Home Attack Performance:")
    for team, row in top_home.iterrows():
        print(f"      • {team}: {row['attack_advantage_factor']:+.1%} (Rank boost: {row['attack_rank_boost']:+.2f})")
else:
    print("❌ Home/away advantage data not found")

# Check 3: Fixture Analyzer
print("\n3️⃣ ENHANCED FIXTURE ANALYZER")
print("-" * 80)
if 'analyzer' in locals():
    print(f"✅ Fixture analyzer initialized successfully")
    print(f"   Teams mapped: {len(analyzer.team_mapping)}")
    print(f"   Total fixtures: {len(analyzer.fixtures_df)}")
    print(f"   Gameweeks: {analyzer.fixtures_df['gameweek'].min()} to {analyzer.fixtures_df['gameweek'].max()}")
else:
    print("❌ Fixture analyzer not initialized")

# Check 4: JSON Exports
print("\n4️⃣ JSON EXPORTS")
print("-" * 80)
export_files = {
    'fixtures.json': 'backend/data/fixture_analysis/fixtures.json',
    'fixture_opportunities.json': 'backend/data/fixture_analysis/fixture_opportunities.json',
    'team_fixture_summary.json': 'backend/data/fixture_analysis/team_fixture_summary.json',
    'all_players.json': 'backend/data/player_trends/all_players.json',
    'player_data.json': 'backend/data/player_trends/player_data.json',
}

all_files_exist = True
for filename, filepath in export_files.items():
    if os.path.exists(filepath):
        size_kb = os.path.getsize(filepath) / 1024
        print(f"✅ {filename:<35} ({size_kb:>7.1f} KB)")
    else:
        print(f"❌ {filename:<35} NOT FOUND")
        all_files_exist = False

# Check 5: Data Validation
print("\n5️⃣ DATA VALIDATION")
print("-" * 80)

# Sample fixture data to verify structure
if os.path.exists('backend/data/fixture_analysis/fixtures.json'):
    with open('backend/data/fixture_analysis/fixtures.json', 'r') as f:
        fixtures = json.load(f)
    
    if fixtures and len(fixtures) > 0:
        sample_fixture = fixtures[0]
        required_keys = ['gameweek', 'fixture', 'home_team', 'away_team']
        home_keys = ['name', 'short_name', 'attacking_fixture_rating', 'defensive_fixture_rating', 'rank']
        
        fixture_ok = all(key in sample_fixture for key in required_keys)
        home_ok = all(key in sample_fixture['home_team'] for key in home_keys)
        
        if fixture_ok and home_ok:
            print(f"✅ Fixture JSON structure valid")
            print(f"   Total fixtures: {len(fixtures)}")
            print(f"   Sample fixture: {sample_fixture['home_team']['name']} vs {sample_fixture['away_team']['name']} (GW{sample_fixture['gameweek']})")
            print(f"   Home Attack Rating: {sample_fixture['home_team']['attacking_fixture_rating']}")
            print(f"   Away Defense Rating: {sample_fixture['away_team']['defensive_fixture_rating']}")
        else:
            print(f"❌ Fixture JSON structure invalid")
            print(f"   Missing keys detected")
else:
    print("❌ fixtures.json not found for validation")

# Final Summary
print("\n" + "="*80)
print("🎯 FEATURE FLOW SUMMARY")
print("="*80)

summary = {
    'Form-Weighted Rankings': 'team_rankings' in locals(),
    'Dynamic Home Advantage': 'home_away_df' in locals(),
    'Fixture Analyzer': 'analyzer' in locals(),
    'JSON Exports': all_files_exist,
}

completed = sum(1 for v in summary.values() if v)
total = len(summary)

for feature, status in summary.items():
    symbol = "✅" if status else "❌"
    print(f"{symbol} {feature}")

print(f"\n📊 COMPLETION: {completed}/{total} features working")

if completed == total:
    print("\n🎉 ALL FEATURES INTEGRATED & FLOWING PROPERLY!")
    print("   ✅ Form weighting applied to team rankings")
    print("   ✅ Dynamic home advantage calculated")
    print("   ✅ Fixture analyzer enhanced with both features")
    print("   ✅ All 5 JSON files exported successfully")
    print("   ✅ No duplicates, clean integration")
else:
    print(f"\n⚠️  {total - completed} feature(s) need attention")

PLayer Trends



In [ ]:
# Convert player data to JSON for faster API performance
import json
import os

def convert_players_to_json():
    """Convert player data from CSV to JSON format"""
    
    # Create player_trends directory
    output_dir = 'backend/data/player_trends'
    os.makedirs(output_dir, exist_ok=True)
    
    print("Converting player data to JSON...")
    
    # Load the CSV data
    df_players = pd.read_csv('fpl-data-stats.csv')
    
    # Fill NaN values first before type conversion
    df_players = df_players.fillna({
        'web_name': 'Unknown',
        'team_name': 'Unknown', 
        'opponent_team_name': 'Unknown',
        'was_home': False,
        'touches': 0,
        'penalty_area_touches': 0,
        'carries_final_third': 0,
        'key_passes': 0,
        'shots': 0,
        'SoT': 0,
        'G': 0,
        'A': 0,
        'CS': 0,
        'GC': 0,
        'minutes': 0,
        'total_points': 0,
        'now_cost': 0,
        'selected_by_percent': 0,
        'xG': 0,
        'xA': 0,
        'xGI': 0,
        'xP': 0,
        'xGC': 0,
        'defensive_contribution': 0
    })
    
    # Convert to native Python types to avoid JSON serialization issues
    df_players = df_players.astype({
        'id': 'int32',
        'element_type': 'int32', 
        'gameweek': 'int32',
        'minutes': 'int32',
        'total_points': 'float32',
        'G': 'int32',
        'A': 'int32',
        'CS': 'int32',
        'shots': 'int32',
        'SoT': 'int32',
        'key_passes': 'int32',
        'touches': 'int32',
        'penalty_area_touches': 'int32',
        'carries_final_third': 'int32',
        'GC': 'int32',
        'now_cost': 'float32',
        'selected_by_percent': 'float32',
        'xG': 'float32',
        'xA': 'float32',
        'xGI': 'float32',
        'xP': 'float32',
        'xGC': 'float32',
        'defensive_contribution': 'float32'
    })
    
    # Convert boolean columns
    df_players['was_home'] = df_players['was_home'].astype(bool)
    
    # Create all_players.json (list of unique players for search)
    latest_gw = df_players.groupby('id')['gameweek'].max()
    unique_players = df_players[df_players.apply(lambda row: row['gameweek'] == latest_gw[row['id']], axis=1)]
    
    players_list = []
    for _, row in unique_players.iterrows():
        players_list.append({
            "id": int(row['id']),
            "name": str(row['web_name']),
            "team": str(row['team_name']),
            "position": int(row['element_type']),
            "cost": round(float(row['now_cost']), 2),
            "ownership": round(float(row['selected_by_percent']), 2)
        })
    
    players_list.sort(key=lambda x: x['name'])
    
    # Save all_players.json
    with open(f'{output_dir}/all_players.json', 'w') as f:
        json.dump({
            "players": players_list,
            "count": len(players_list)
        }, f, indent=2)
    
    print(f"✅ Saved {len(players_list)} players to all_players.json")
    
    # Create player_data.json (all gameweek data organized by player)
    player_data = {}
    
    for player_name in df_players['web_name'].dropna().unique():
        player_gw_data = df_players[df_players['web_name'] == player_name].copy()
        
        if player_gw_data.empty:
            continue
            
        # Sort by gameweek
        player_gw_data = player_gw_data.sort_values('gameweek')
        
        # Get player info from most recent gameweek
        player_info = player_gw_data.iloc[-1]
        
        # Calculate form (last 5 GWs)
        last_5_gws = player_gw_data.tail(5)
        form_stats = {
            "avg_points": round(float(last_5_gws['total_points'].mean()), 1),
            "avg_minutes": round(float(last_5_gws['minutes'].mean()), 0),
            "games_played": int(len(last_5_gws))
        }
        
        # Gameweek data
        gameweeks = []
        for _, row in player_gw_data.iterrows():
            gameweeks.append({
                "gameweek": int(row['gameweek']),
                "opponent": str(row['opponent_team_name']),
                "was_home": bool(row['was_home']),
                "total_points": float(row['total_points']),
                "minutes": int(row['minutes']),
                "goals": int(row['G']),
                "assists": int(row['A']),
                "clean_sheets": int(row['CS']),
                "xG": round(float(row['xG']), 2),
                "xA": round(float(row['xA']), 2),
                "xGI": round(float(row['xGI']), 2),
                "xP": round(float(row['xP']), 2),
                "shots": int(row['shots']),
                "shots_on_target": int(row['SoT']),
                "key_passes": int(row['key_passes']),
                "touches": int(row['touches']),
                "penalty_area_touches": int(row['penalty_area_touches']),
                "carries_final_third": int(row['carries_final_third']),
                "defensive_contribution": round(float(row['defensive_contribution']), 2),
                "xGC": round(float(row['xGC']), 2),
                "goals_conceded": int(row['GC'])
            })
        
        # Total stats
        total_minutes = int(player_gw_data['minutes'].sum())
        total_stats = {
            "games_played": int(len(player_gw_data)),
            "total_points": int(player_gw_data['total_points'].sum()),
            "total_goals": int(player_gw_data['G'].sum()),
            "total_assists": int(player_gw_data['A'].sum()),
            "total_xG": round(float(player_gw_data['xG'].sum()), 2),
            "total_xA": round(float(player_gw_data['xA'].sum()), 2),
            "total_xGI": round(float(player_gw_data['xGI'].sum()), 2),
            "total_xP": round(float(player_gw_data['xP'].sum()), 2),
            "total_minutes": total_minutes,
            "total_shots": int(player_gw_data['shots'].sum()),
            "total_key_passes": int(player_gw_data['key_passes'].sum())
        }
        
        # Per-90 stats
        per90_stats = {
            "points_per_90": round((total_stats["total_points"] * 90) / max(total_minutes, 1), 2),
            "goals_per_90": round((total_stats["total_goals"] * 90) / max(total_minutes, 1), 2),
            "assists_per_90": round((total_stats["total_assists"] * 90) / max(total_minutes, 1), 2),
            "xG_per_90": round((total_stats["total_xG"] * 90) / max(total_minutes, 1), 2),
            "xA_per_90": round((total_stats["total_xA"] * 90) / max(total_minutes, 1), 2),
            "xGI_per_90": round((total_stats["total_xGI"] * 90) / max(total_minutes, 1), 2),
            "shots_per_90": round((total_stats["total_shots"] * 90) / max(total_minutes, 1), 2),
            "key_passes_per_90": round((total_stats["total_key_passes"] * 90) / max(total_minutes, 1), 2)
        }
        
        # Store player data
        player_data[player_name] = {
            "player_name": str(player_name),
            "team": str(player_info['team_name']),
            "position": int(player_info['element_type']),
            "web_name": str(player_info['web_name']),
            "cost": round(float(player_info['now_cost']), 2),
            "ownership": round(float(player_info['selected_by_percent']), 2),
            "form": form_stats,
            "total_stats": total_stats,
            "per90_stats": per90_stats,
            "gameweeks": gameweeks
        }
    
    # Save player_data.json
    with open(f'{output_dir}/player_data.json', 'w') as f:
        json.dump(player_data, f, indent=2)
    
    print(f"✅ Saved detailed data for {len(player_data)} players to player_data.json")
    print(f"📁 Files created in: {output_dir}/")
    
    return len(players_list), len(player_data)

# Run the conversion
player_count, detail_count = convert_players_to_json()
print(f"\n🎯 Conversion complete:")
print(f"   - {player_count} players in search index")  
print(f"   - {detail_count} players with detailed stats")